# Energiemanagement Case Study - Investitionsanalyse für Kraftwerke

## Lernziele

In dieser Case Study analysieren Sie Investitionsentscheidungen für drei Kraftwerkstechnologien:
- Windkraft (Onshore)
- Biomassekraftwerk
- Batteriespeicher

Sie lernen dabei:
1. Die Berechnung von Kapitalwerten (NPV) unter Unsicherheit
2. Den Umgang mit stochastischen Preisschwankungen mittels Monte-Carlo-Simulation
3. Die Bewertung von Investitionsportfolios unter Risiko-Rendite-Aspekten
4. Die Durchführung von Szenarioanalysen für strategische Entscheidungen

## Grundlegende Modellstruktur

### Technische und ökonomische Parameter

Die nachfolgende Implementierung definiert die Basisparameter für unsere Analyse. Jede Technologie wird durch spezifische technische und wirtschaftliche Kennzahlen charakterisiert:

- **CAPEX**: Investitionskosten in €/kW installierter Leistung
- **FOM**: Fixe Betriebskosten pro Jahr in €/kW
- **VOM**: Variable Betriebskosten in €/MWh produzierter Energie
- **Lebensdauer**: Betriebsjahre der Anlage
- **Volllaststunden**: Jährliche Betriebsstunden bei Nennleistung

Diese Parameter basieren auf aktuellen Marktwerten und technischen Spezifikationen.

In [ ]:
# Cell 1: Setup & Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Styling
widget_layout = widgets.Layout(width='80%')
slider_style = {'description_width': '150px'}

# Farbschema
colors = {
    'wind': '#1f77b4',
    'biomass': '#2ca02c',
    'battery': '#ff7f0e'
}

In [ ]:
# Cell 2: Basis NPV-Funktionen und Parameter
class EnergyInvestmentModel:
    def __init__(self):
        # Technologie-Parameter
        self.tech_params = {
            'wind': {
                'name': 'Wind Onshore',
                'capex': 1197,  # €/kW
                'fom': 36,      # €/kW/a
                'vom': 1,       # €/MWh
                'lifetime': 22,
                'full_load_hours': 2452.5
            },
            'biomass': {
                'name': 'Biomass',
                'capex': 2000,  # €/kW
                'fom': 80,      # €/kW/a
                'vom': 2,       # €/MWh
                'lifetime': 40,
                'efficiency': 0.42,
                'full_load_hours': 6542
            },
            'battery': {
                'name': 'Battery',
                'capex': 504,   # €/kW (inverter + storage)
                'fom': 2.31,    # €/kW/a
                'lifetime': 20,
                'efficiency': 0.9,
                'cycles_per_year': 300,
                'energy_to_power': 2
            }
        }

        # Standard-Parameter
        self.fuel_heating_value = 5.278  # MWh/t
        self.price_spread_battery = 0.2  # 20% für Arbitrage

    def calculate_npv(self, tech, elec_price, bio_price=70.92, discount_rate=0.05):
        """Berechnet NPV für eine Technologie"""
        params = self.tech_params[tech]

        if tech == 'wind':
            return self._npv_wind(params, elec_price, discount_rate)
        elif tech == 'biomass':
            return self._npv_biomass(params, elec_price, bio_price, discount_rate)
        elif tech == 'battery':
            return self._npv_battery(params, elec_price, discount_rate)

    def _npv_wind(self, params, elec_price, discount_rate):
        cashflows = [-params['capex']]

        for year in range(1, params['lifetime'] + 1):
            revenue = params['full_load_hours'] * elec_price
            costs = params['fom'] + (params['vom'] * params['full_load_hours'] / 1000)
            annual_cf = revenue - costs
            discounted_cf = annual_cf / (1 + discount_rate) ** year
            cashflows.append(discounted_cf)

        return sum(cashflows)

    def _npv_biomass(self, params, elec_price, bio_price, discount_rate):
        cashflows = [-params['capex']]

        bio_cost_per_kwh_th = bio_price / (self.fuel_heating_value * 1000)
        bio_cost_per_kwh_el = bio_cost_per_kwh_th / params['efficiency']

        for year in range(1, params['lifetime'] + 1):
            revenue = params['full_load_hours'] * elec_price
            fuel_costs = params['full_load_hours'] * bio_cost_per_kwh_el
            other_costs = params['fom'] + (params['vom'] * params['full_load_hours'] / 1000)
            annual_cf = revenue - fuel_costs - other_costs
            discounted_cf = annual_cf / (1 + discount_rate) ** year
            cashflows.append(discounted_cf)

        return sum(cashflows)

    def _npv_battery(self, params, elec_price, discount_rate):
        cashflows = [-params['capex']]

        for year in range(1, params['lifetime'] + 1):
            daily_spread = elec_price * self.price_spread_battery
            revenue = params['cycles_per_year'] * params['energy_to_power'] * daily_spread * params['efficiency']
            costs = params['fom']
            annual_cf = revenue - costs
            discounted_cf = annual_cf / (1 + discount_rate) ** year
            cashflows.append(discounted_cf)

        return sum(cashflows)

    def get_cashflow_details(self, tech, elec_price, bio_price=70.92, discount_rate=0.05):
        """Gibt detaillierte Cashflows zurück für Visualisierung"""
        params = self.tech_params[tech]
        years = list(range(params['lifetime'] + 1))
        cashflows = []
        discounted_cashflows = []

        # Jahr 0
        cashflows.append(-params['capex'])
        discounted_cashflows.append(-params['capex'])

        # Berechnung je nach Technologie
        for year in range(1, params['lifetime'] + 1):
            if tech == 'wind':
                revenue = params['full_load_hours'] * elec_price
                costs = params['fom'] + (params['vom'] * params['full_load_hours'] / 1000)
            elif tech == 'biomass':
                bio_cost_per_kwh_el = (bio_price / (self.fuel_heating_value * 1000)) / params['efficiency']
                revenue = params['full_load_hours'] * elec_price
                costs = params['fom'] + (params['vom'] * params['full_load_hours'] / 1000) + (params['full_load_hours'] * bio_cost_per_kwh_el)
            elif tech == 'battery':
                daily_spread = elec_price * self.price_spread_battery
                revenue = params['cycles_per_year'] * params['energy_to_power'] * daily_spread * params['efficiency']
                costs = params['fom']

            annual_cf = revenue - costs
            cashflows.append(annual_cf)
            discounted_cashflows.append(annual_cf / (1 + discount_rate) ** year)

        return years[:len(cashflows)], cashflows, discounted_cashflows

# Modell initialisieren
model = EnergyInvestmentModel()

## Aufgabe 1: Interaktive NPV-Berechnung

### Zielsetzung

In diesem Abschnitt berechnen Sie den Kapitalwert (NPV) für jede Technologie basierend auf den Mittelwerten der Strom- und Biomassepreise. Die interaktiven Slider ermöglichen es Ihnen, die Sensitivität der Ergebnisse auf Parameteränderungen zu untersuchen.

### Bedienung

1. **Strompreis-Slider**: Variieren Sie den durchschnittlichen Strompreis und beobachten Sie die Auswirkungen auf alle drei Technologien
2. **Biomassepreis-Slider**: Dieser Parameter beeinflusst nur das Biomassekraftwerk (Brennstoffkosten)
3. **Diskontierungsrate**: Zeigt den Einfluss des Zeitwerts des Geldes auf langfristige Investitionen

### Interpretation der Ergebnisse

Ein positiver NPV bedeutet, dass die Investition bei den gewählten Parametern wirtschaftlich vorteilhaft ist. Die Visualisierungen zeigen:
- **NPV-Vergleich**: Direkter Vergleich der Wirtschaftlichkeit
- **Cashflow-Verlauf**: Zeitpunkt der Amortisation (Break-Even)
- **Sensitivitätsanalyse**: Kritische Preisschwellen für die Rentabilität

In [ ]:
# Cell 3: Interaktiver Parameter-Explorer für Aufgabe 1

# Widget-Container für bessere Organisation
class ParameterExplorer:
    def __init__(self, model):
        self.model = model
        self.setup_widgets()
        self.setup_layout()

    def setup_widgets(self):
        # Slider für Parameter
        self.elec_price_slider = widgets.FloatSlider(
            value=0.06976,
            min=0.03,
            max=0.12,
            step=0.001,
            description='Strompreis [€/kWh]:',
            style=slider_style,
            layout=widget_layout
        )

        self.bio_price_slider = widgets.FloatSlider(
            value=70.92,
            min=30,
            max=120,
            step=1,
            description='Biomassepreis [€/t]:',
            style=slider_style,
            layout=widget_layout
        )

        self.discount_slider = widgets.FloatSlider(
            value=0.05,
            min=0.01,
            max=0.15,
            step=0.01,
            description='Diskontierungsrate:',
            style=slider_style,
            layout=widget_layout
        )

        # Ausgabe-Widgets
        self.output_npv = widgets.Output()
        self.output_plot = widgets.Output()
        self.output_sensitivity = widgets.Output()

        # Checkboxen für Technologien
        self.tech_checkboxes = {
            'wind': widgets.Checkbox(value=True, description='Wind Onshore'),
            'biomass': widgets.Checkbox(value=True, description='Biomasse'),
            'battery': widgets.Checkbox(value=True, description='Batterie')
        }

        # Plot-Auswahl
        self.plot_selector = widgets.Dropdown(
            options=['NPV Vergleich', 'Cashflow-Verlauf', 'Sensitivität Strompreis', 'Sensitivität Biomasse'],
            value='NPV Vergleich',
            description='Visualisierung:'
        )

    def setup_layout(self):
        # Event-Handler
        self.elec_price_slider.observe(self.update_all, 'value')
        self.bio_price_slider.observe(self.update_all, 'value')
        self.discount_slider.observe(self.update_all, 'value')
        self.plot_selector.observe(self.update_all, 'value')

        for checkbox in self.tech_checkboxes.values():
            checkbox.observe(self.update_all, 'value')

        # Layout
        self.parameter_box = widgets.VBox([
            widgets.HTML('<h3>Parameter-Einstellungen</h3>'),
            self.elec_price_slider,
            self.bio_price_slider,
            self.discount_slider
        ])

        self.tech_box = widgets.VBox([
            widgets.HTML('<h3>Technologien</h3>'),
            widgets.HBox(list(self.tech_checkboxes.values()))
        ])

        self.viz_box = widgets.VBox([
            widgets.HTML('<h3>Visualisierung</h3>'),
            self.plot_selector,
            self.output_plot
        ])

        self.results_box = widgets.VBox([
            widgets.HTML('<h3>NPV-Ergebnisse</h3>'),
            self.output_npv
        ])

        self.main_layout = widgets.VBox([
            self.parameter_box,
            self.tech_box,
            widgets.HBox([self.results_box, self.viz_box])
        ])

    def calculate_npvs(self):
        """Berechnet NPVs für alle aktivierten Technologien"""
        results = {}
        active_techs = [tech for tech, cb in self.tech_checkboxes.items() if cb.value]

        for tech in active_techs:
            npv = self.model.calculate_npv(
                tech,
                self.elec_price_slider.value,
                self.bio_price_slider.value,
                self.discount_slider.value
            )
            results[tech] = npv

        return results

    def update_all(self, change):
        """Update-Funktion für alle Widgets"""
        self.update_npv_display()
        self.update_plot()

    def update_npv_display(self):
        """Aktualisiert die NPV-Anzeige"""
        with self.output_npv:
            clear_output(wait=True)

            results = self.calculate_npvs()

            # Erstelle Tabelle
            df = pd.DataFrame([
                {'Technologie': self.model.tech_params[tech]['name'],
                 'NPV [€/kW]': f'{npv:,.2f}',
                 'Status': '✅ Profitabel' if npv > 0 else '❌ Unprofitabel'}
                for tech, npv in results.items()
            ])

            display(df.style.set_properties(**{
                'text-align': 'left',
                'font-size': '12pt'
            }).hide(axis='index'))

    def update_plot(self):
        """Aktualisiert die Visualisierung"""
        with self.output_plot:
            clear_output(wait=True)

            if self.plot_selector.value == 'NPV Vergleich':
                self.plot_npv_comparison()
            elif self.plot_selector.value == 'Cashflow-Verlauf':
                self.plot_cashflows()
            elif self.plot_selector.value == 'Sensitivität Strompreis':
                self.plot_sensitivity_electricity()
            elif self.plot_selector.value == 'Sensitivität Biomasse':
                self.plot_sensitivity_biomass()

    def plot_npv_comparison(self):
        """NPV Balkendiagramm"""
        results = self.calculate_npvs()

        fig = go.Figure()

        for tech, npv in results.items():
            color = colors[tech]
            fig.add_trace(go.Bar(
                x=[self.model.tech_params[tech]['name']],
                y=[npv],
                name=self.model.tech_params[tech]['name'],
                marker_color=color,
                text=f'{npv:,.0f}',
                textposition='outside'
            ))

        fig.update_layout(
            title='NPV-Vergleich der Technologien',
            xaxis_title='Technologie',
            yaxis_title='NPV [€/kW]',
            showlegend=False,
            height=600,
                    width=600
        )

        fig.add_hline(y=0, line_dash="dash", line_color="red", opacity=0.5)
        fig.show()

    def plot_cashflows(self):
        """Cashflow-Verlauf über Zeit"""
        fig = go.Figure()

        active_techs = [tech for tech, cb in self.tech_checkboxes.items() if cb.value]

        for tech in active_techs:
            years, cf, dcf = self.model.get_cashflow_details(
                tech,
                self.elec_price_slider.value,
                self.bio_price_slider.value,
                self.discount_slider.value
            )

            fig.add_trace(go.Scatter(
                x=years,
                y=np.cumsum(dcf),
                mode='lines+markers',
                name=self.model.tech_params[tech]['name'],
                line=dict(color=colors[tech], width=2)
            ))

        fig.update_layout(
            title='Kumulierte diskontierte Cashflows',
            xaxis_title='Jahr',
            yaxis_title='Kumulierter DCF [€/kW]',
            hovermode='x unified',
            height=600,
                    width=600
        )

        fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.3)
        fig.show()

    def plot_sensitivity_electricity(self):
        """Sensitivitätsanalyse Strompreis"""
        prices = np.linspace(0.03, 0.12, 50)

        fig = go.Figure()

        active_techs = [tech for tech, cb in self.tech_checkboxes.items() if cb.value]

        for tech in active_techs:
            npvs = [self.model.calculate_npv(
                tech, p, self.bio_price_slider.value, self.discount_slider.value
            ) for p in prices]

            fig.add_trace(go.Scatter(
                x=prices,
                y=npvs,
                mode='lines',
                name=self.model.tech_params[tech]['name'],
                line=dict(color=colors[tech], width=2)
            ))

        # Aktuelle Position markieren
        fig.add_vline(
            x=self.elec_price_slider.value,
            line_dash="dash",
            line_color="gray",
            annotation_text="Aktuell"
        )

        fig.update_layout(
            title='Sensitivität: Strompreis',
            xaxis_title='Strompreis [€/kWh]',
            yaxis_title='NPV [€/kW]',
            hovermode='x unified',
            height=600,
            width=600
        )

        fig.add_hline(y=0, line_dash="dash", line_color="red", opacity=0.5)
        fig.show()

    def plot_sensitivity_biomass(self):
        """2D Heatmap für Biomasse"""
        elec_prices = np.linspace(0.04, 0.10, 25)
        bio_prices = np.linspace(40, 100, 25)

        npv_matrix = np.zeros((len(bio_prices), len(elec_prices)))

        for i, bp in enumerate(bio_prices):
            for j, ep in enumerate(elec_prices):
                npv_matrix[i, j] = self.model.calculate_npv(
                    'biomass', ep, bp, self.discount_slider.value
                )

        fig = go.Figure(data=go.Heatmap(
            x=elec_prices,
            y=bio_prices,
            z=npv_matrix,
            colorscale='RdYlGn',
            zmid=0,
            text=np.round(npv_matrix, 0),
            texttemplate='%{text}',
            textfont={"size": 8},
            colorbar=dict(title="NPV [€/kW]")
        ))

        # Aktuelle Position
        fig.add_trace(go.Scatter(
            x=[self.elec_price_slider.value],
            y=[self.bio_price_slider.value],
            mode='markers',
            marker=dict(size=15, color='blue', symbol='x'),
            name='Aktuell',
            showlegend=True
        ))

        fig.update_layout(
            title='Biomasse NPV: Strom- vs. Biomassepreis',
            xaxis_title='Strompreis [€/kWh]',
            yaxis_title='Biomassepreis [€/t]',
            height=500
        )

        fig.show()

    def display(self):
        """Zeigt das komplette Widget an"""
        display(self.main_layout)
        self.update_all(None)



In [ ]:
# Cell 4: Explorer starten
explorer = ParameterExplorer(model)
explorer.display()

## Sensitivitätsanalyse mittels Tornado-Diagramm

### Konzept

Das Tornado-Diagramm zeigt, welche Parameter den größten Einfluss auf den NPV haben. Jeder Balken repräsentiert die NPV-Änderung bei einer Variation des Parameters um ±20%.

### Interpretation

- **Lange Balken**: Parameter mit hohem Einfluss auf das Ergebnis
- **Kurze Balken**: Parameter mit geringem Einfluss
- **Reihenfolge**: Von oben nach unten abnehmender Einfluss

Diese Analyse hilft zu identifizieren, auf welche Parameter bei der Investitionsentscheidung besonders geachtet werden muss.

In [ ]:
# Cell 5: Tornado-Diagramm für Sensitivitätsanalyse

def create_tornado_diagram():
    """Erstellt ein Tornado-Diagramm für alle Technologien"""

    # Basis-Parameter
    base_elec = 0.06976
    base_bio = 70.92
    base_discount = 0.05

    # Variationsbereiche (±20%)
    variations = {
        'Strompreis': {'param': 'elec', 'base': base_elec, 'low': base_elec * 0.8, 'high': base_elec * 1.2},
        'Biomassepreis': {'param': 'bio', 'base': base_bio, 'low': base_bio * 0.8, 'high': base_bio * 1.2},
        'Diskontrate': {'param': 'disc', 'base': base_discount, 'low': base_discount * 0.8, 'high': base_discount * 1.2}
    }

    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=['Wind Onshore', 'Biomasse', 'Batterie']
    )

    for col, tech in enumerate(['wind', 'biomass', 'battery'], 1):
        # Basis NPV
        base_npv = model.calculate_npv(tech, base_elec, base_bio, base_discount)

        impacts = []
        labels = []

        for param_name, var in variations.items():
            if var['param'] == 'elec':
                low_npv = model.calculate_npv(tech, var['low'], base_bio, base_discount)
                high_npv = model.calculate_npv(tech, var['high'], base_bio, base_discount)
            elif var['param'] == 'bio':
                low_npv = model.calculate_npv(tech, base_elec, var['low'], base_discount)
                high_npv = model.calculate_npv(tech, base_elec, var['high'], base_discount)
            elif var['param'] == 'disc':
                low_npv = model.calculate_npv(tech, base_elec, base_bio, var['low'])
                high_npv = model.calculate_npv(tech, base_elec, base_bio, var['high'])

            impact = high_npv - low_npv
            impacts.append(impact)
            labels.append(param_name)

        # Sortieren nach Impact
        sorted_data = sorted(zip(impacts, labels), reverse=True)
        sorted_impacts, sorted_labels = zip(*sorted_data)

        fig.add_trace(
            go.Bar(
                y=sorted_labels,
                x=sorted_impacts,
                orientation='h',
                marker_color=colors[tech],
                showlegend=False
            ),
            row=1, col=col
        )

    fig.update_layout(
        title_text='Tornado-Diagramm: Sensitivität der Parameter (±20% Variation)',
        height=600,
        width=1800
    )

    fig.show()

# Tornado-Diagramm Button
tornado_button = widgets.Button(description='Tornado-Diagramm anzeigen')
tornado_output = widgets.Output()

def on_tornado_click(b):
    with tornado_output:
        clear_output()
        create_tornado_diagram()

tornado_button.on_click(on_tornado_click)

display(widgets.VBox([tornado_button, tornado_output]))

## Aufgabe 2 & 3: Monte-Carlo Simulation und Visualisierung

### Methodischer Ansatz

Die Monte-Carlo Simulation berücksichtigt die Unsicherheit in den Eingangsparametern. Anstatt mit festen Werten zu rechnen, werden die Preise aus Normalverteilungen gezogen:

- **Strompreis**: Normalverteilt mit Mittelwert μ und Standardabweichung σ
- **Biomassepreis**: Ebenfalls normalverteilt mit eigenen Parametern

### Durchführung

1. Es werden 100 (oder mehr) zufällige Preiskombinationen generiert
2. Für jede Kombination wird der NPV aller Technologien berechnet
3. Die Ergebnisse zeigen die Verteilung möglicher NPV-Werte

### Visualisierungsoptionen

- **Boxplot**: Zeigt Median, Quartile und Ausreißer
- **Violinplot**: Visualisiert zusätzlich die Verteilungsform
- **Histogramm**: Häufigkeitsverteilung der NPV-Werte
- **Kombinierte Ansicht**: Mehrere Perspektiven gleichzeitig

### Statistische Auswertung (Aufgabe 4)

Die berechneten Kennzahlen umfassen:
- **Mittelwert**: Erwarteter NPV bei Unsicherheit
- **Standardabweichung**: Maß für das Risiko
- **Perzentile**: Wahrscheinlichkeitsaussagen (z.B. "mit 95% Wahrscheinlichkeit liegt der NPV über X")

In [ ]:
# Cell 6: Monte-Carlo Simulation Control Panel

class MonteCarloSimulator:
    def __init__(self, model):
        self.model = model
        self.simulation_results = None
        self.price_data = None
        self.setup_widgets()
        self.setup_layout()

    def setup_widgets(self):
        # Simulations-Parameter
        self.n_simulations_slider = widgets.IntSlider(
            value=100,
            min=10,
            max=1000,
            step=10,
            description='Anzahl Simulationen:',
            style=slider_style,
            layout=widget_layout
        )

        # Preis-Parameter mit Standardabweichung
        self.elec_mean = widgets.FloatSlider(
            value=0.06976,
            min=0.03,
            max=0.12,
            step=0.001,
            description='Strom Mittelwert [€/kWh]:',
            style=slider_style,
            layout=widget_layout
        )

        self.elec_std = widgets.FloatSlider(
            value=0.00244,
            min=0.0005,
            max=0.01,
            step=0.0001,
            description='Strom Std.Abw. [€/kWh]:',
            style=slider_style,
            layout=widget_layout
        )

        self.bio_mean = widgets.FloatSlider(
            value=70.92,
            min=30,
            max=120,
            step=1,
            description='Biomasse Mittelwert [€/t]:',
            style=slider_style,
            layout=widget_layout
        )

        self.bio_std = widgets.FloatSlider(
            value=11.76,
            min=1,
            max=30,
            step=0.5,
            description='Biomasse Std.Abw. [€/t]:',
            style=slider_style,
            layout=widget_layout
        )

        self.discount_rate = widgets.FloatSlider(
            value=0.05,
            min=0.01,
            max=0.15,
            step=0.01,
            description='Diskontierungsrate:',
            style=slider_style,
            layout=widget_layout
        )

        # Seed für Reproduzierbarkeit
        self.seed_input = widgets.IntText(
            value=42,
            description='Random Seed:',
            style={'description_width': '100px'}
        )

        self.use_seed = widgets.Checkbox(
            value=True,
            description='Seed verwenden'
        )

        # Buttons
        self.run_button = widgets.Button(
            description='Simulation starten',
            button_style='primary',
            icon='play'
        )

        self.export_button = widgets.Button(
            description='Ergebnisse exportieren',
            button_style='info',
            icon='download',
            disabled=True
        )

        # Progress Bar
        self.progress = widgets.IntProgress(
            value=0,
            min=0,
            max=100,
            description='Fortschritt:',
            style={'description_width': '100px'},
            layout=widgets.Layout(width='90%')
        )

        # Visualisierungs-Optionen
        self.plot_type = widgets.RadioButtons(
            options=['Boxplot', 'Violin Plot', 'Histogramm', 'Kombiniert'],
            value='Boxplot',
            description='Plot-Typ:'
        )

        self.show_outliers = widgets.Checkbox(
            value=True,
            description='Ausreißer anzeigen'
        )

        self.show_mean = widgets.Checkbox(
            value=True,
            description='Mittelwert anzeigen'
        )

        # Technologie-Filter
        self.tech_filter = {
            'wind': widgets.Checkbox(value=True, description='Wind'),
            'biomass': widgets.Checkbox(value=True, description='Biomasse'),
            'battery': widgets.Checkbox(value=True, description='Batterie')
        }

        # Output Areas
        self.output_status = widgets.Output()
        self.output_plot = widgets.Output()
        self.output_stats = widgets.Output()

    def setup_layout(self):
        # Event Handler
        self.run_button.on_click(self.run_simulation)
        self.export_button.on_click(self.export_results)
        self.plot_type.observe(self.update_plot, 'value')
        self.show_outliers.observe(self.update_plot, 'value')
        self.show_mean.observe(self.update_plot, 'value')

        for checkbox in self.tech_filter.values():
            checkbox.observe(self.update_plot, 'value')

        # Layout Struktur
        self.param_box = widgets.VBox([
            widgets.HTML('<h3>📊 Preis-Parameter (Normalverteilung)</h3>'),
            widgets.HTML('<b>Strompreis:</b>'),
            self.elec_mean,
            self.elec_std,
            widgets.HTML('<b>Biomassepreis:</b>'),
            self.bio_mean,
            self.bio_std,
            widgets.HTML('<b>Weitere Parameter:</b>'),
            self.discount_rate
        ])

        self.sim_control = widgets.VBox([
            widgets.HTML('<h3>🎲 Simulations-Einstellungen</h3>'),
            self.n_simulations_slider,
            widgets.HBox([self.use_seed, self.seed_input]),
            widgets.HBox([self.run_button, self.export_button]),
            self.progress,
            self.output_status
        ])

        self.viz_control = widgets.VBox([
            widgets.HTML('<h3>📈 Visualisierungs-Optionen</h3>'),
            self.plot_type,
            widgets.HBox([self.show_outliers, self.show_mean]),
            widgets.HBox(list(self.tech_filter.values()))
        ])

        self.results_area = widgets.VBox([
            widgets.HTML('<h3>📊 Ergebnisse</h3>'),
            widgets.Tab([self.output_plot, self.output_stats])
        ])

        # Tab-Titel setzen
        self.results_area.children[1].set_title(0, 'Visualisierung')
        self.results_area.children[1].set_title(1, 'Statistiken')

        # Hauptlayout
        self.main_layout = widgets.VBox([
            widgets.HBox([
                widgets.VBox([self.param_box, self.sim_control], layout=widgets.Layout(width='40%')),
                widgets.VBox([self.viz_control, self.results_area], layout=widgets.Layout(width='60%'))
            ])
        ])

    def run_simulation(self, button):
        """Führt die Monte-Carlo Simulation durch"""
        with self.output_status:
            clear_output(wait=True)
            print("🚀 Starte Simulation...")

        # Seed setzen wenn gewünscht
        if self.use_seed.value:
            np.random.seed(self.seed_input.value)

        n_sims = self.n_simulations_slider.value

        # Arrays für Ergebnisse
        results = {
            'wind': [],
            'biomass': [],
            'battery': []
        }

        # Preise generieren
        elec_prices = np.random.normal(self.elec_mean.value, self.elec_std.value, n_sims)
        bio_prices = np.random.normal(self.bio_mean.value, self.bio_std.value, n_sims)

        # Speichere Preisdaten für späteren Export
        self.price_data = pd.DataFrame({
            'electricity_price': elec_prices,
            'biomass_price': bio_prices
        })

        # Simulation mit Progress Bar
        for i in range(n_sims):
            # NPV Berechnungen
            results['wind'].append(
                self.model.calculate_npv('wind', elec_prices[i], bio_prices[i], self.discount_rate.value)
            )
            results['biomass'].append(
                self.model.calculate_npv('biomass', elec_prices[i], bio_prices[i], self.discount_rate.value)
            )
            results['battery'].append(
                self.model.calculate_npv('battery', elec_prices[i], bio_prices[i], self.discount_rate.value)
            )

            # Progress Update
            self.progress.value = int((i + 1) / n_sims * 100)

        # Ergebnisse speichern
        self.simulation_results = pd.DataFrame(results)

        with self.output_status:
            print(f"✅ Simulation abgeschlossen! {n_sims} Durchläufe berechnet.")

        # Export-Button aktivieren
        self.export_button.disabled = False

        # Plots und Statistiken aktualisieren
        self.update_plot(None)
        self.update_statistics()

    def update_plot(self, change):
        """Aktualisiert die Visualisierung"""
        if self.simulation_results is None:
            return

        with self.output_plot:
            clear_output(wait=True)

            # Gefilterte Daten
            active_techs = [tech for tech, cb in self.tech_filter.items() if cb.value]
            data = self.simulation_results[active_techs]

            if self.plot_type.value == 'Boxplot':
                self.create_boxplot(data)
            elif self.plot_type.value == 'Violin Plot':
                self.create_violin_plot(data)
            elif self.plot_type.value == 'Histogramm':
                self.create_histogram(data)
            elif self.plot_type.value == 'Kombiniert':
                self.create_combined_plot(data)

    def create_boxplot(self, data):
        """Erstellt einen interaktiven Boxplot"""
        fig = go.Figure()

        for col in data.columns:
            fig.add_trace(go.Box(
                y=data[col],
                name=self.model.tech_params[col]['name'],
                boxpoints='outliers' if self.show_outliers.value else False,
                marker_color=colors[col],
                boxmean=self.show_mean.value
            ))

        fig.update_layout(
            title=f'NPV-Verteilung ({self.n_simulations_slider.value} Simulationen)',
            yaxis_title='NPV [€/kW]',
            showlegend=False,
            height=500
        )

        fig.add_hline(y=0, line_dash="dash", line_color="red", opacity=0.5)
        fig.show()

    def create_violin_plot(self, data):
        """Erstellt einen Violin Plot"""
        fig = go.Figure()

        for col in data.columns:
            fig.add_trace(go.Violin(
                y=data[col],
                name=self.model.tech_params[col]['name'],
                box_visible=True,
                meanline_visible=self.show_mean.value,
                marker_color=colors[col]
            ))

        fig.update_layout(
            title=f'NPV-Verteilung (Violin Plot)',
            yaxis_title='NPV [€/kW]',
            showlegend=False,
            height=500
        )

        fig.add_hline(y=0, line_dash="dash", line_color="red", opacity=0.5)
        fig.show()

    def create_histogram(self, data):
        """Erstellt überlappende Histogramme"""
        fig = go.Figure()

        for col in data.columns:
            fig.add_trace(go.Histogram(
                x=data[col],
                name=self.model.tech_params[col]['name'],
                opacity=0.7,
                marker_color=colors[col],
                nbinsx=30
            ))

        fig.update_layout(
            title='NPV-Verteilung (Histogramm)',
            xaxis_title='NPV [€/kW]',
            yaxis_title='Häufigkeit',
            barmode='overlay',
            height=500
        )

        fig.add_vline(x=0, line_dash="dash", line_color="red", opacity=0.5)
        fig.show()

    def create_combined_plot(self, data):
        """Erstellt eine kombinierte Ansicht"""
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Boxplot', 'Verteilungsdichte', 'Risiko-Rendite', 'Konfidenzintervalle'),
            specs=[[{"type": "box"}, {"type": "scatter"}],
                   [{"type": "scatter"}, {"type": "bar"}]]
        )

        # 1. Boxplot
        for i, col in enumerate(data.columns):
            fig.add_trace(
                go.Box(y=data[col], name=self.model.tech_params[col]['name'],
                      marker_color=colors[col], showlegend=False),
                row=1, col=1
            )

        # 2. Verteilungsdichte
        for col in data.columns:
            hist, bins = np.histogram(data[col], bins=50, density=True)
            bin_centers = (bins[:-1] + bins[1:]) / 2
            fig.add_trace(
                go.Scatter(x=bin_centers, y=hist, name=self.model.tech_params[col]['name'],
                          line=dict(color=colors[col]), showlegend=True),
                row=1, col=2
            )

        # 3. Risiko-Rendite
        means = data.mean()
        stds = data.std()
        for col in data.columns:
            fig.add_trace(
                go.Scatter(x=[stds[col]], y=[means[col]],
                          mode='markers+text',
                          marker=dict(size=15, color=colors[col]),
                          text=[self.model.tech_params[col]['name']],
                          textposition="top center",
                          showlegend=False),
                row=2, col=1
            )

        # 4. Konfidenzintervalle
        confidence_levels = [0.90, 0.95, 0.99]
        for i, col in enumerate(data.columns):
            intervals = []
            for cl in confidence_levels:
                lower = np.percentile(data[col], (1-cl)/2 * 100)
                upper = np.percentile(data[col], (1+cl)/2 * 100)
                intervals.append(upper - lower)

            fig.add_trace(
                go.Bar(x=[f'{cl*100:.0f}%' for cl in confidence_levels],
                      y=intervals,
                      name=self.model.tech_params[col]['name'],
                      marker_color=colors[col],
                      showlegend=False),
                row=2, col=2
            )

        # Layout Updates
        fig.update_xaxes(title_text="", row=1, col=1)
        fig.update_xaxes(title_text="NPV [€/kW]", row=1, col=2)
        fig.update_xaxes(title_text="Risiko (Std.Abw.) [€/kW]", row=2, col=1)
        fig.update_xaxes(title_text="Konfidenzniveau", row=2, col=2)

        fig.update_yaxes(title_text="NPV [€/kW]", row=1, col=1)
        fig.update_yaxes(title_text="Dichte", row=1, col=2)
        fig.update_yaxes(title_text="Erwartungswert [€/kW]", row=2, col=1)
        fig.update_yaxes(title_text="Intervallbreite [€/kW]", row=2, col=2)

        fig.update_layout(height=800, showlegend=True, title_text="Kombinierte Analyse")
        fig.show()

    def update_statistics(self):
        """Berechnet und zeigt detaillierte Statistiken"""
        if self.simulation_results is None:
            return

        with self.output_stats:
            clear_output(wait=True)

            # Basis-Statistiken
            stats_df = pd.DataFrame({
                'Technologie': [self.model.tech_params[col]['name'] for col in self.simulation_results.columns],
                'Mittelwert': self.simulation_results.mean().values,
                'Median': self.simulation_results.median().values,
                'Std.Abw.': self.simulation_results.std().values,
                'Min': self.simulation_results.min().values,
                'Max': self.simulation_results.max().values,
                'Skewness': self.simulation_results.skew().values,
                'Kurtosis': self.simulation_results.kurtosis().values
            })

            # Wahrscheinlichkeiten
            prob_positive = []
            prob_above_1000 = []

            for col in self.simulation_results.columns:
                prob_positive.append((self.simulation_results[col] > 0).mean() * 100)
                prob_above_1000.append((self.simulation_results[col] > 1000).mean() * 100)

            stats_df['P(NPV>0)'] = prob_positive
            stats_df['P(NPV>1000)'] = prob_above_1000

            # Formatierung
            display(widgets.HTML('<h4>📊 Statistische Kennzahlen</h4>'))
            display(stats_df.style.format({
                'Mittelwert': '{:.2f}',
                'Median': '{:.2f}',
                'Std.Abw.': '{:.2f}',
                'Min': '{:.2f}',
                'Max': '{:.2f}',
                'Skewness': '{:.3f}',
                'Kurtosis': '{:.3f}',
                'P(NPV>0)': '{:.1f}%',
                'P(NPV>1000)': '{:.1f}%'
            }).background_gradient(subset=['Mittelwert', 'P(NPV>0)'], cmap='RdYlGn'))

            # Perzentile
            display(widgets.HTML('<h4>📈 Perzentile</h4>'))
            percentiles = [5, 10, 25, 50, 75, 90, 95]
            perc_data = {}

            for col in self.simulation_results.columns:
                perc_data[self.model.tech_params[col]['name']] = [
                    np.percentile(self.simulation_results[col], p) for p in percentiles
                ]

            perc_df = pd.DataFrame(perc_data, index=[f'{p}%' for p in percentiles])
            display(perc_df.style.format('{:.2f}').background_gradient(cmap='YlOrRd', axis=0))

            # Korrelationsmatrix
            if len(self.simulation_results.columns) > 1:
                display(widgets.HTML('<h4>🔗 Korrelationsmatrix</h4>'))
                corr_matrix = self.simulation_results.corr()

                fig = go.Figure(data=go.Heatmap(
                    z=corr_matrix.values,
                    x=[self.model.tech_params[col]['name'] for col in corr_matrix.columns],
                    y=[self.model.tech_params[col]['name'] for col in corr_matrix.columns],
                    text=np.round(corr_matrix.values, 3),
                    texttemplate='%{text}',
                    colorscale='RdBu',
                    zmid=0
                ))

                fig.update_layout(height=300, title='Korrelation der NPV-Werte')
                fig.show()

    def export_results(self, button):
        """Exportiert die Simulationsergebnisse"""
        if self.simulation_results is None:
            return

        # Timestamp für eindeutige Dateinamen
        timestamp = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')

        # Excel-Export mit mehreren Sheets
        with pd.ExcelWriter(f'monte_carlo_results_{timestamp}.xlsx') as writer:
            # NPV Ergebnisse
            self.simulation_results.to_excel(writer, sheet_name='NPV_Results', index=True)

            # Preis-Daten
            self.price_data.to_excel(writer, sheet_name='Price_Data', index=True)

            # Statistiken
            stats_df = pd.DataFrame({
                'Metric': ['Mean', 'Std', 'Min', 'Max', 'P(NPV>0)', 'P(NPV>1000)'],
                'Wind': [
                    self.simulation_results['wind'].mean(),
                    self.simulation_results['wind'].std(),
                    self.simulation_results['wind'].min(),
                    self.simulation_results['wind'].max(),
                    (self.simulation_results['wind'] > 0).mean() * 100,
                    (self.simulation_results['wind'] > 1000).mean() * 100
                ],
                'Biomass': [
                    self.simulation_results['biomass'].mean(),
                    self.simulation_results['biomass'].std(),
                    self.simulation_results['biomass'].min(),
                    self.simulation_results['biomass'].max(),
                    (self.simulation_results['biomass'] > 0).mean() * 100,
                    (self.simulation_results['biomass'] > 1000).mean() * 100
                ],
                'Battery': [
                    self.simulation_results['battery'].mean(),
                    self.simulation_results['battery'].std(),
                    self.simulation_results['battery'].min(),
                    self.simulation_results['battery'].max(),
                    (self.simulation_results['battery'] > 0).mean() * 100,
                    (self.simulation_results['battery'] > 1000).mean() * 100
                ]
            })
            stats_df.to_excel(writer, sheet_name='Statistics', index=False)

            # Parameter
            params_df = pd.DataFrame({
                'Parameter': ['Electricity Price Mean', 'Electricity Price Std',
                             'Biomass Price Mean', 'Biomass Price Std',
                             'Discount Rate', 'Number of Simulations', 'Random Seed'],
                'Value': [self.elec_mean.value, self.elec_std.value,
                         self.bio_mean.value, self.bio_std.value,
                         self.discount_rate.value, self.n_simulations_slider.value,
                         self.seed_input.value if self.use_seed.value else 'None']
            })
            params_df.to_excel(writer, sheet_name='Parameters', index=False)

        with self.output_status:
            print(f"✅ Ergebnisse exportiert: monte_carlo_results_{timestamp}.xlsx")

    def display(self):
        """Zeigt das komplette Widget an"""
        display(self.main_layout)

# Cell 7: Monte-Carlo Simulator starten
mc_simulator = MonteCarloSimulator(model)
mc_simulator.display()

## Erweiterte statistische Analyse

### Wahrscheinlichkeitsrechner

Dieser Abschnitt beantwortet praktische Fragen wie:
- "Wie wahrscheinlich ist es, dass die Investition profitabel ist?"
- "Wie hoch ist die Wahrscheinlichkeit, einen NPV über 1000 €/kW zu erreichen?"

### Value at Risk (VaR)

Der VaR gibt an, welcher NPV-Wert mit einer bestimmten Wahrscheinlichkeit nicht unterschritten wird. Beispiel: Ein VaR(95%) von 500 €/kW bedeutet, dass mit 95% Wahrscheinlichkeit der NPV mindestens 500 €/kW beträgt.

### Conditional Value at Risk (CVaR)

Der CVaR beantwortet die Frage: "Wenn es schlecht läuft (schlechteste 5% der Fälle), wie hoch ist dann der durchschnittliche Verlust?"

In [ ]:
# Cell 8: Quick Analysis für Aufgabe 4

class QuickAnalysisDashboard:
    def __init__(self, simulator):
        self.simulator = simulator
        self.setup_widgets()

    def setup_widgets(self):
        # Wahrscheinlichkeitsrechner
        self.threshold_slider = widgets.FloatSlider(
            value=0,
            min=-2000,
            max=3000,
            step=50,
            description='NPV Schwellwert:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='50%')
        )

        self.prob_output = widgets.Output()

        # Value at Risk
        self.var_level = widgets.FloatSlider(
            value=0.95,
            min=0.90,
            max=0.99,
            step=0.01,
            description='VaR Level:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='50%')
        )

        self.var_output = widgets.Output()

        # Layout
        self.layout = widgets.VBox([
            widgets.HTML('<h3>🎯 Schnellanalyse der Ergebnisse</h3>'),
            widgets.HBox([
                widgets.VBox([
                    widgets.HTML('<b>Wahrscheinlichkeitsrechner</b>'),
                    self.threshold_slider,
                    self.prob_output
                ]),
                widgets.VBox([
                    widgets.HTML('<b>Value at Risk (VaR)</b>'),
                    self.var_level,
                    self.var_output
                ])
            ])
        ])

        # Event Handler
        self.threshold_slider.observe(self.update_probability, 'value')
        self.var_level.observe(self.update_var, 'value')

    def update_probability(self, change):
        """Berechnet P(NPV > threshold) für alle Technologien"""
        if self.simulator.simulation_results is None:
            return

        with self.prob_output:
            clear_output(wait=True)

            threshold = self.threshold_slider.value

            for tech in self.simulator.simulation_results.columns:
                prob = (self.simulator.simulation_results[tech] > threshold).mean() * 100
                tech_name = self.simulator.model.tech_params[tech]['name']

                # Farbcodierung
                if prob > 80:
                    color = 'green'
                elif prob > 50:
                    color = 'orange'
                else:
                    color = 'red'

                print(f"{tech_name}: {prob:.1f}%", end="  ")

                # Visualisierung als Mini-Bar
                bar_length = int(prob / 5)  # max 20 Zeichen
                bar = '█' * bar_length + '░' * (20 - bar_length)
                print(f"[{bar}]")

    def update_var(self, change):
        """Berechnet Value at Risk"""
        if self.simulator.simulation_results is None:
            return

        with self.var_output:
            clear_output(wait=True)

            level = self.var_level.value

            for tech in self.simulator.simulation_results.columns:
                var = np.percentile(self.simulator.simulation_results[tech], (1-level)*100)
                cvar = self.simulator.simulation_results[tech][
                    self.simulator.simulation_results[tech] <= var
                ].mean()

                tech_name = self.simulator.model.tech_params[tech]['name']
                print(f"{tech_name}:")
                print(f"  VaR({level:.0%}): {var:.2f} €/kW")
                print(f"  CVaR({level:.0%}): {cvar:.2f} €/kW")
                print()

    def display(self):
        display(self.layout)
        self.update_probability(None)
        self.update_var(None)

# Quick Analysis anzeigen (nur wenn Simulation durchgeführt wurde)
if hasattr(mc_simulator, 'simulation_results') and mc_simulator.simulation_results is not None:
    quick_analysis = QuickAnalysisDashboard(mc_simulator)
    quick_analysis.display()
else:
    print("⚠️ Bitte führen Sie zuerst eine Monte-Carlo Simulation durch!")

## Aufgabe 5-7: Portfolio-Optimierung

### Grundkonzept der Diversifikation

Durch die Kombination verschiedener Technologien in einem Portfolio kann das Gesamtrisiko reduziert werden, ohne proportional auf Rendite verzichten zu müssen. Dies basiert auf dem Prinzip der Diversifikation aus der Portfoliotheorie.

### Portfolio-Definition

Jedes Portfolio wird durch die prozentualen Anteile der drei Technologien definiert:
- Die Summe muss 100% ergeben
- Jede Technologie kann zwischen 0% und 100% gewichtet werden

### Analyse-Metriken

1. **Portfolio-NPV**: Gewichteter Durchschnitt der Einzel-NPVs
2. **Portfolio-Risiko**: Berücksichtigt Korrelationen zwischen den Technologien
3. **Sharpe Ratio**: Rendite pro Risikoeinheit (höher = besser)

### Optimierungsziele

Sie können Portfolios nach verschiedenen Kriterien optimieren:
- **Maximaler NPV**: Höchste erwartete Rendite
- **Minimales Risiko**: Geringste Schwankung
- **Maximale Sharpe Ratio**: Bestes Risiko-Rendite-Verhältnis

In [ ]:
# Cell 9: Portfolio-Analyse Dashboard

class PortfolioAnalyzer:
    def __init__(self, model, mc_simulator):
        self.model = model
        self.mc_simulator = mc_simulator
        self.portfolio_results = None
        self.setup_widgets()
        self.setup_layout()

    def setup_widgets(self):
        # Portfolio-Definition Widgets
        self.portfolio_tabs = widgets.Tab()
        self.portfolio_configs = []

        # 4 vordefinierte Portfolios
        default_portfolios = [
            {'name': 'Portfolio 1', 'wind': 0.5, 'biomass': 0.3, 'battery': 0.2},
            {'name': 'Portfolio 2', 'wind': 0.7, 'biomass': 0.2, 'battery': 0.1},
            {'name': 'Portfolio 3', 'wind': 0.3, 'biomass': 0.3, 'battery': 0.4},
            {'name': 'Portfolio 4', 'wind': 0.4, 'biomass': 0.4, 'battery': 0.2}
        ]

        # Erstelle Widgets für jedes Portfolio
        for i, default in enumerate(default_portfolios):
            portfolio_widget = self.create_portfolio_widget(i+1, default)
            self.portfolio_configs.append(portfolio_widget)

        # Custom Portfolio
        self.custom_portfolio = self.create_custom_portfolio_widget()

        # Optimierungs-Widgets
        self.optimization_type = widgets.RadioButtons(
            options=['Maximiere NPV', 'Minimiere Risiko', 'Maximiere Sharpe Ratio'],
            value='Maximiere NPV',
            description='Ziel:'
        )

        self.constraint_type = widgets.RadioButtons(
            options=['Keine', 'Max. Risiko', 'Min. NPV'],
            value='Keine',
            description='Beschränkung:'
        )

        self.constraint_value = widgets.FloatSlider(
            value=500,
            min=0,
            max=2000,
            step=50,
            description='Wert:',
            style={'description_width': '50px'}
        )

        # Buttons
        self.calculate_button = widgets.Button(
            description='Portfolios berechnen',
            button_style='primary',
            icon='calculator'
        )

        self.optimize_button = widgets.Button(
            description='Portfolio optimieren',
            button_style='success',
            icon='chart-line'
        )

        # Output Areas
        self.output_summary = widgets.Output()
        self.output_visualization = widgets.Output()
        self.output_comparison = widgets.Output()
        self.output_optimization = widgets.Output()

    def create_portfolio_widget(self, number, default_values):
        """Erstellt Widgets für ein Portfolio"""
        name_input = widgets.Text(
            value=default_values['name'],
            description='Name:',
            style={'description_width': '50px'},
            layout=widgets.Layout(width='200px')
        )

        wind_slider = widgets.FloatSlider(
            value=default_values['wind'],
            min=0,
            max=1,
            step=0.05,
            description='Wind:',
            style={'description_width': '60px'},
            readout_format='.0%'
        )

        biomass_slider = widgets.FloatSlider(
            value=default_values['biomass'],
            min=0,
            max=1,
            step=0.05,
            description='Biomasse:',
            style={'description_width': '60px'},
            readout_format='.0%'
        )

        battery_slider = widgets.FloatSlider(
            value=default_values['battery'],
            min=0,
            max=1,
            step=0.05,
            description='Batterie:',
            style={'description_width': '60px'},
            readout_format='.0%'
        )

        # Summen-Label
        sum_label = widgets.Label(value=f'Summe: {sum(default_values.values()) - 1:.0%}')

        # Auto-Normalisierung
        normalize_button = widgets.Button(
            description='Normalisieren',
            button_style='info',
            layout=widgets.Layout(width='100px')
        )

        def update_sum(*args):
            total = wind_slider.value + biomass_slider.value + battery_slider.value
            sum_label.value = f'Summe: {total:.0%}'
            sum_label.style = {'description_width': '50px'}
            if abs(total - 1.0) < 0.001:
                sum_label.layout.border = '2px solid green'
            else:
                sum_label.layout.border = '2px solid red'

        def normalize(*args):
            total = wind_slider.value + biomass_slider.value + battery_slider.value
            if total > 0:
                wind_slider.value = wind_slider.value / total
                biomass_slider.value = biomass_slider.value / total
                battery_slider.value = battery_slider.value / total

        # Event Handler
        wind_slider.observe(update_sum, 'value')
        biomass_slider.observe(update_sum, 'value')
        battery_slider.observe(update_sum, 'value')
        normalize_button.on_click(normalize)

        # Initial update
        update_sum()

        # Container
        widget_box = widgets.VBox([
            name_input,
            wind_slider,
            biomass_slider,
            battery_slider,
            widgets.HBox([sum_label, normalize_button])
        ])

        return {
            'box': widget_box,
            'name': name_input,
            'wind': wind_slider,
            'biomass': biomass_slider,
            'battery': battery_slider
        }

    def create_custom_portfolio_widget(self):
        """Erstellt Widget für benutzerdefiniertes Portfolio"""
        return self.create_portfolio_widget(5, {
            'name': 'Custom Portfolio',
            'wind': 0.33,
            'biomass': 0.33,
            'battery': 0.34
        })

    def setup_layout(self):
        # Event Handler
        self.calculate_button.on_click(self.calculate_portfolios)
        self.optimize_button.on_click(self.optimize_portfolio)

        # Portfolio Tabs zusammenbauen
        portfolio_boxes = [config['box'] for config in self.portfolio_configs]
        portfolio_boxes.append(self.custom_portfolio['box'])

        self.portfolio_tabs.children = portfolio_boxes
        for i in range(4):
            self.portfolio_tabs.set_title(i, f'Portfolio {i+1}')
        self.portfolio_tabs.set_title(4, 'Custom')

        # Hauptlayout
        self.input_section = widgets.VBox([
            widgets.HTML('<h3>📊 Portfolio-Definition</h3>'),
            self.portfolio_tabs,
            self.calculate_button
        ])

        self.optimization_section = widgets.VBox([
            widgets.HTML('<h3>🎯 Portfolio-Optimierung</h3>'),
            self.optimization_type,
            widgets.HBox([self.constraint_type, self.constraint_value]),
            self.optimize_button,
            self.output_optimization
        ])

        self.results_section = widgets.Tab([
            self.output_summary,
            self.output_visualization,
            self.output_comparison
        ])
        self.results_section.set_title(0, 'Zusammenfassung')
        self.results_section.set_title(1, 'Visualisierung')
        self.results_section.set_title(2, 'Vergleich')

        self.main_layout = widgets.VBox([
            widgets.HBox([
                self.input_section,
                self.optimization_section
            ]),
            widgets.HTML('<h3>📈 Ergebnisse</h3>'),
            self.results_section
        ])

    def get_portfolio_weights(self):
        """Sammelt alle Portfolio-Gewichtungen"""
        all_configs = self.portfolio_configs + [self.custom_portfolio]
        portfolios = {}

        for i, config in enumerate(all_configs):
            name = config['name'].value
            weights = {
                'wind': config['wind'].value,
                'biomass': config['biomass'].value,
                'battery': config['battery'].value
            }
            # Nur Portfolios mit Summe ≈ 1 berücksichtigen
            if abs(sum(weights.values()) - 1.0) < 0.01:
                portfolios[name] = weights

        return portfolios

    def calculate_portfolio_metrics(self, weights, returns_data):
        """Berechnet NPV und Risiko für ein Portfolio"""
        # Portfolio-Returns für jede Simulation
        portfolio_returns = (
            returns_data['wind'] * weights['wind'] +
            returns_data['biomass'] * weights['biomass'] +
            returns_data['battery'] * weights['battery']
        )

        mean_return = portfolio_returns.mean()
        std_return = portfolio_returns.std()

        return mean_return, std_return, portfolio_returns

    def calculate_portfolios(self, button):
        """Berechnet alle Portfolios"""
        # Prüfe ob Simulationsdaten vorhanden
        if self.mc_simulator.simulation_results is None:
            with self.output_summary:
                clear_output()
                print("⚠️ Bitte führen Sie zuerst eine Monte-Carlo Simulation durch!")
            return

        # Sammle Portfolio-Definitionen
        portfolios = self.get_portfolio_weights()

        if not portfolios:
            with self.output_summary:
                clear_output()
                print("⚠️ Keine gültigen Portfolios definiert (Summe muss 100% sein)!")
            return

        # Berechne Metriken
        self.portfolio_results = {}

        for name, weights in portfolios.items():
            mean_npv, std_npv, returns = self.calculate_portfolio_metrics(
                weights, self.mc_simulator.simulation_results
            )

            self.portfolio_results[name] = {
                'weights': weights,
                'mean_npv': mean_npv,
                'std_npv': std_npv,
                'sharpe_ratio': mean_npv / std_npv if std_npv > 0 else 0,
                'returns': returns,
                'prob_positive': (returns > 0).mean() * 100,
                'var_95': np.percentile(returns, 5),
                'cvar_95': returns[returns <= np.percentile(returns, 5)].mean()
            }

        # Aktualisiere Anzeigen
        self.update_summary()
        self.update_visualization()
        self.update_comparison()

    def update_summary(self):
        """Zeigt Portfolio-Zusammenfassung"""
        with self.output_summary:
            clear_output(wait=True)

            # Erstelle Übersichtstabelle
            summary_data = []

            for name, metrics in self.portfolio_results.items():
                summary_data.append({
                    'Portfolio': name,
                    'NPV (Erwartung)': f"{metrics['mean_npv']:.2f}",
                    'Risiko (Std)': f"{metrics['std_npv']:.2f}",
                    'Sharpe Ratio': f"{metrics['sharpe_ratio']:.3f}",
                    'P(NPV>0)': f"{metrics['prob_positive']:.1f}%",
                    'VaR(95%)': f"{metrics['var_95']:.2f}",
                    'Wind': f"{metrics['weights']['wind']:.0%}",
                    'Biomasse': f"{metrics['weights']['biomass']:.0%}",
                    'Batterie': f"{metrics['weights']['battery']:.0%}"
                })

            df = pd.DataFrame(summary_data)

            # Styling
            display(widgets.HTML('<h4>Portfolio-Übersicht</h4>'))
            display(df.style\
                .background_gradient(subset=['NPV (Erwartung)'], cmap='RdYlGn')\
                .background_gradient(subset=['Risiko (Std)'], cmap='RdYlGn_r')\
                .background_gradient(subset=['Sharpe Ratio'], cmap='viridis'))

            # Beste Portfolios identifizieren
            display(widgets.HTML('<h4>Top-Portfolios</h4>'))

            best_npv = max(self.portfolio_results.items(), key=lambda x: x[1]['mean_npv'])
            best_sharpe = max(self.portfolio_results.items(), key=lambda x: x[1]['sharpe_ratio'])
            lowest_risk = min(self.portfolio_results.items(), key=lambda x: x[1]['std_npv'])

            print(f"📈 Höchster NPV: {best_npv[0]} ({best_npv[1]['mean_npv']:.2f} €/kW)")
            print(f"⚖️ Bestes Risiko-Rendite: {best_sharpe[0]} (Sharpe: {best_sharpe[1]['sharpe_ratio']:.3f})")
            print(f"🛡️ Geringstes Risiko: {lowest_risk[0]} (Std: {lowest_risk[1]['std_npv']:.2f} €/kW)")

    def update_visualization(self):
        """Erstellt interaktive Visualisierungen"""
        with self.output_visualization:
            clear_output(wait=True)

            # 1. Risiko-Rendite-Diagramm
            fig = make_subplots(
                rows=2, cols=2,
                subplot_titles=('Risiko-Rendite-Profil', 'Portfolio-Zusammensetzung',
                              'Verteilungen', 'Korrelation mit Einzeltechnologien'),
                specs=[[{"type": "scatter"}, {"type": "pie"}],
                       [{"type": "box"}, {"type": "scatter"}]]
            )

            # Subplot 1: Risiko-Rendite
            # Einzeltechnologien
            for tech in ['wind', 'biomass', 'battery']:
                tech_returns = self.mc_simulator.simulation_results[tech]
                fig.add_trace(
                    go.Scatter(
                        x=[tech_returns.std()],
                        y=[tech_returns.mean()],
                        mode='markers',
                        name=self.model.tech_params[tech]['name'],
                        marker=dict(size=15, color=colors[tech], symbol='circle'),
                        showlegend=True
                    ),
                    row=1, col=1
                )

            # Portfolios
            for name, metrics in self.portfolio_results.items():
                fig.add_trace(
                    go.Scatter(
                        x=[metrics['std_npv']],
                        y=[metrics['mean_npv']],
                        mode='markers+text',
                        name=name,
                        text=[name],
                        textposition='top center',
                        marker=dict(size=12, symbol='diamond'),
                        showlegend=True
                    ),
                    row=1, col=1
                )

            # Subplot 2: Portfolio-Zusammensetzung (für das beste Portfolio)
            best_portfolio = max(self.portfolio_results.items(),
                               key=lambda x: x[1]['sharpe_ratio'])

            fig.add_trace(
                go.Pie(
                    labels=['Wind', 'Biomasse', 'Batterie'],
                    values=[
                        best_portfolio[1]['weights']['wind'],
                        best_portfolio[1]['weights']['biomass'],
                        best_portfolio[1]['weights']['battery']
                    ],
                    marker_colors=[colors['wind'], colors['biomass'], colors['battery']],
                    title=best_portfolio[0]
                ),
                row=1, col=2
            )

            # Subplot 3: Verteilungen
            for name, metrics in self.portfolio_results.items():
                fig.add_trace(
                    go.Box(
                        y=metrics['returns'],
                        name=name,
                        boxmean=True
                    ),
                    row=2, col=1
                )

            # Subplot 4: Effizienzgrenze
            # Generiere zusätzliche Portfolios für glatte Kurve
            n_points = 50
            weights_range = np.linspace(0, 1, n_points)
            frontier_risk = []
            frontier_return = []

            for w1 in weights_range:
                for w2 in np.linspace(0, 1-w1, int((1-w1)*n_points)):
                    w3 = 1 - w1 - w2
                    if w3 >= 0:
                        weights = {'wind': w1, 'biomass': w2, 'battery': w3}
                        mean_ret, std_ret, _ = self.calculate_portfolio_metrics(
                            weights, self.mc_simulator.simulation_results
                        )
                        frontier_risk.append(std_ret)
                        frontier_return.append(mean_ret)

            # Efficient Frontier
            fig.add_trace(
                go.Scatter(
                    x=frontier_risk,
                    y=frontier_return,
                    mode='markers',
                    marker=dict(size=2, color='lightgray'),
                    name='Mögliche Portfolios',
                    showlegend=False
                ),
                row=2, col=2
            )

            # Layout
            fig.update_xaxes(title_text="Risiko (Std) [€/kW]", row=1, col=1)
            fig.update_yaxes(title_text="Erwarteter NPV [€/kW]", row=1, col=1)
            fig.update_yaxes(title_text="NPV [€/kW]", row=2, col=1)
            fig.update_xaxes(title_text="Risiko (Std) [€/kW]", row=2, col=2)
            fig.update_yaxes(title_text="Erwarteter NPV [€/kW]", row=2, col=2)

            fig.update_layout(height=800, showlegend=True,
                            title_text="Portfolio-Analyse Dashboard")
            fig.show()

    def update_comparison(self):
        """Vergleicht Portfolios mit Einzeltechnologien"""
        with self.output_comparison:
            clear_output(wait=True)

            # Vorbereitung der Daten
            comparison_data = []

            # Einzeltechnologien
            for tech in ['wind', 'biomass', 'battery']:
                tech_returns = self.mc_simulator.simulation_results[tech]
                comparison_data.append({
                    'Name': self.model.tech_params[tech]['name'],
                    'Typ': 'Einzeltechnologie',
                    'NPV': tech_returns.mean(),
                    'Risiko': tech_returns.std(),
                    'Sharpe': tech_returns.mean() / tech_returns.std(),
                    'P(NPV>0)': (tech_returns > 0).mean() * 100,
                    'VaR(95%)': np.percentile(tech_returns, 5)
                })

            # Portfolios
            for name, metrics in self.portfolio_results.items():
                comparison_data.append({
                    'Name': name,
                    'Typ': 'Portfolio',
                    'NPV': metrics['mean_npv'],
                    'Risiko': metrics['std_npv'],
                    'Sharpe': metrics['sharpe_ratio'],
                    'P(NPV>0)': metrics['prob_positive'],
                    'VaR(95%)': metrics['var_95']
                })

            df_comparison = pd.DataFrame(comparison_data)

            # Radar Chart
            fig = go.Figure()

            categories = ['NPV', 'Risiko⁻¹', 'Sharpe', 'P(NPV>0)', '-VaR(95%)']

            # Normalisiere Daten für Radar Chart (0-1 Skala)
            def normalize_column(col):
                return (col - col.min()) / (col.max() - col.min())

            for _, row in df_comparison.iterrows():
                values = [
                    normalize_column(df_comparison['NPV'])[row.name],
                    1 - normalize_column(df_comparison['Risiko'])[row.name],  # Invertiert
                    normalize_column(df_comparison['Sharpe'])[row.name],
                    row['P(NPV>0)'] / 100,  # Bereits in %
                    1 - normalize_column(df_comparison['VaR(95%)'].abs())[row.name]  # Invertiert
                ]

                fig.add_trace(go.Scatterpolar(
                    r=values,
                    theta=categories,
                    fill='toself',
                    name=row['Name'],
                    line=dict(width=2 if row['Typ'] == 'Portfolio' else 1,
                             dash='solid' if row['Typ'] == 'Portfolio' else 'dash')
                ))

            fig.update_layout(
                polar=dict(
                    radialaxis=dict(
                        visible=True,
                        range=[0, 1]
                    )),
                showlegend=True,
                title="Multidimensionaler Vergleich (normalisiert)",
                height=500
            )

            fig.show()

            # Tabelle
            display(widgets.HTML('<h4>Detaillierter Vergleich</h4>'))
            display(df_comparison.style.format({
                'NPV': '{:.2f}',
                'Risiko': '{:.2f}',
                'Sharpe': '{:.3f}',
                'P(NPV>0)': '{:.1f}%',
                'VaR(95%)': '{:.2f}'
            }).background_gradient(subset=['NPV', 'Sharpe'], cmap='RdYlGn')\
              .background_gradient(subset=['Risiko'], cmap='RdYlGn_r'))

    def optimize_portfolio(self, button):
        """Optimiert Portfolio basierend auf gewähltem Kriterium"""
        with self.output_optimization:
            clear_output(wait=True)

            if self.mc_simulator.simulation_results is None:
                print("⚠️ Keine Simulationsdaten vorhanden!")
                return

            print("🔄 Optimiere Portfolio...")

            # Optimierungsfunktion
            from scipy.optimize import minimize

            def objective(weights):
                w_dict = {'wind': weights[0], 'biomass': weights[1], 'battery': weights[2]}
                mean_npv, std_npv, _ = self.calculate_portfolio_metrics(
                    w_dict, self.mc_simulator.simulation_results
                )

                if self.optimization_type.value == 'Maximiere NPV':
                    return -mean_npv
                elif self.optimization_type.value == 'Minimiere Risiko':
                    return std_npv
                else:  # Maximiere Sharpe Ratio
                    return -mean_npv / std_npv if std_npv > 0 else 0

            # Constraints
            constraints = [{'type': 'eq', 'fun': lambda w: sum(w) - 1}]

            if self.constraint_type.value == 'Max. Risiko':
                def risk_constraint(weights):
                    w_dict = {'wind': weights[0], 'biomass': weights[1], 'battery': weights[2]}
                    _, std_npv, _ = self.calculate_portfolio_metrics(
                        w_dict, self.mc_simulator.simulation_results
                    )
                    return self.constraint_value.value - std_npv

                constraints.append({'type': 'ineq', 'fun': risk_constraint})

            elif self.constraint_type.value == 'Min. NPV':
                def npv_constraint(weights):
                    w_dict = {'wind': weights[0], 'biomass': weights[1], 'battery': weights[2]}
                    mean_npv, _, _ = self.calculate_portfolio_metrics(
                        w_dict, self.mc_simulator.simulation_results
                    )
                    return mean_npv - self.constraint_value.value

                constraints.append({'type': 'ineq', 'fun': npv_constraint})

            # Optimierung
            bounds = [(0, 1), (0, 1), (0, 1)]
            initial_guess = [1/3, 1/3, 1/3]

            result = minimize(objective, initial_guess, method='SLSQP',
                            bounds=bounds, constraints=constraints)

            if result.success:
                optimal_weights = {
                    'wind': result.x[0],
                    'biomass': result.x[1],
                    'battery': result.x[2]
                }

                mean_npv, std_npv, returns = self.calculate_portfolio_metrics(
                    optimal_weights, self.mc_simulator.simulation_results
                )

                print("✅ Optimierung erfolgreich!\n")
                print(f"Optimales Portfolio:")
                print(f"  Wind:     {optimal_weights['wind']:.1%}")
                print(f"  Biomasse: {optimal_weights['biomass']:.1%}")
                print(f"  Batterie: {optimal_weights['battery']:.1%}")
                print(f"\nErgebnisse:")
                print(f"  Erwarteter NPV: {mean_npv:.2f} €/kW")
                print(f"  Risiko (Std):   {std_npv:.2f} €/kW")
                print(f"  Sharpe Ratio:   {mean_npv/std_npv:.3f}")
                print(f"  P(NPV>0):       {(returns > 0).mean()*100:.1f}%")

                # Visualisierung
                fig = go.Figure(data=[
                    go.Bar(
                        x=['Wind', 'Biomasse', 'Batterie'],
                        y=[optimal_weights['wind'], optimal_weights['biomass'],
                           optimal_weights['battery']],
                        marker_color=[colors['wind'], colors['biomass'], colors['battery']],
                        text=[f"{w:.1%}" for w in optimal_weights.values()],
                        textposition='auto'
                    )
                ])

                fig.update_layout(
                    title=f'Optimales Portfolio ({self.optimization_type.value})',
                    yaxis_title='Anteil',
                    yaxis_tickformat='.0%',
                    height=600,
                    width=600
                )

                fig.show()

                # Füge optimales Portfolio zu Custom hinzu
                self.custom_portfolio['name'].value = 'Optimal'
                self.custom_portfolio['wind'].value = optimal_weights['wind']
                self.custom_portfolio['biomass'].value = optimal_weights['biomass']
                self.custom_portfolio['battery'].value = optimal_weights['battery']

            else:
                print("❌ Optimierung fehlgeschlagen!")
                print(f"Grund: {result.message}")
# Cell 9: Portfolio-Analyse Dashboard

class PortfolioAnalyzer:
    def __init__(self, model, mc_simulator):
        self.model = model
        self.mc_simulator = mc_simulator
        self.portfolio_results = None
        self.setup_widgets()
        self.setup_layout()

    def setup_widgets(self):
        # Portfolio-Definition Widgets
        self.portfolio_tabs = widgets.Tab()
        self.portfolio_configs = []

        # 4 vordefinierte Portfolios
        default_portfolios = [
            {'name': 'Portfolio 1', 'wind': 0.5, 'biomass': 0.3, 'battery': 0.2},
            {'name': 'Portfolio 2', 'wind': 0.7, 'biomass': 0.2, 'battery': 0.1},
            {'name': 'Portfolio 3', 'wind': 0.3, 'biomass': 0.3, 'battery': 0.4},
            {'name': 'Portfolio 4', 'wind': 0.4, 'biomass': 0.4, 'battery': 0.2}
        ]

        # Erstelle Widgets für jedes Portfolio
        for i, default in enumerate(default_portfolios):
            portfolio_widget = self.create_portfolio_widget(i+1, default)
            self.portfolio_configs.append(portfolio_widget)

        # Custom Portfolio
        self.custom_portfolio = self.create_custom_portfolio_widget()

        # Optimierungs-Widgets
        self.optimization_type = widgets.RadioButtons(
            options=['Maximiere NPV', 'Minimiere Risiko', 'Maximiere Sharpe Ratio'],
            value='Maximiere NPV',
            description='Ziel:'
        )

        self.constraint_type = widgets.RadioButtons(
            options=['Keine', 'Max. Risiko', 'Min. NPV'],
            value='Keine',
            description='Beschränkung:'
        )

        self.constraint_value = widgets.FloatSlider(
            value=500,
            min=0,
            max=2000,
            step=50,
            description='Wert:',
            style={'description_width': '50px'}
        )

        # Buttons
        self.calculate_button = widgets.Button(
            description='Portfolios berechnen',
            button_style='primary',
            icon='calculator'
        )

        self.optimize_button = widgets.Button(
            description='Portfolio optimieren',
            button_style='success',
            icon='chart-line'
        )

        # Output Areas
        self.output_summary = widgets.Output()
        self.output_visualization = widgets.Output()
        self.output_comparison = widgets.Output()
        self.output_optimization = widgets.Output()

    def create_portfolio_widget(self, number, default_values):
        """Erstellt Widgets für ein Portfolio"""
        name_input = widgets.Text(
            value=default_values['name'],
            description='Name:',
            style={'description_width': '50px'},
            layout=widgets.Layout(width='200px')
        )

        wind_slider = widgets.FloatSlider(
            value=default_values['wind'],
            min=0,
            max=1,
            step=0.05,
            description='Wind:',
            style={'description_width': '60px'},
            readout_format='.0%'
        )

        biomass_slider = widgets.FloatSlider(
            value=default_values['biomass'],
            min=0,
            max=1,
            step=0.05,
            description='Biomasse:',
            style={'description_width': '60px'},
            readout_format='.0%'
        )

        battery_slider = widgets.FloatSlider(
            value=default_values['battery'],
            min=0,
            max=1,
            step=0.05,
            description='Batterie:',
            style={'description_width': '60px'},
            readout_format='.0%'
        )

        # Summen-Label - Korrigiert: Summiere nur die numerischen Werte
        initial_sum = default_values['wind'] + default_values['biomass'] + default_values['battery']
        sum_label = widgets.Label(value=f'Summe: {initial_sum:.0%}')

        # Auto-Normalisierung
        normalize_button = widgets.Button(
            description='Normalisieren',
            button_style='info',
            layout=widgets.Layout(width='100px')
        )

        def update_sum(*args):
            total = wind_slider.value + biomass_slider.value + battery_slider.value
            sum_label.value = f'Summe: {total:.0%}'
            sum_label.style = {'description_width': '50px'}
            if abs(total - 1.0) < 0.001:
                sum_label.layout.border = '2px solid green'
            else:
                sum_label.layout.border = '2px solid red'

        def normalize(*args):
            total = wind_slider.value + biomass_slider.value + battery_slider.value
            if total > 0:
                wind_slider.value = wind_slider.value / total
                biomass_slider.value = biomass_slider.value / total
                battery_slider.value = battery_slider.value / total

        # Event Handler
        wind_slider.observe(update_sum, 'value')
        biomass_slider.observe(update_sum, 'value')
        battery_slider.observe(update_sum, 'value')
        normalize_button.on_click(normalize)

        # Initial update
        update_sum()

        # Container
        widget_box = widgets.VBox([
            name_input,
            wind_slider,
            biomass_slider,
            battery_slider,
            widgets.HBox([sum_label, normalize_button])
        ])

        return {
            'box': widget_box,
            'name': name_input,
            'wind': wind_slider,
            'biomass': biomass_slider,
            'battery': battery_slider
        }

    def create_custom_portfolio_widget(self):
        """Erstellt Widget für benutzerdefiniertes Portfolio"""
        return self.create_portfolio_widget(5, {
            'name': 'Custom Portfolio',
            'wind': 0.33,
            'biomass': 0.33,
            'battery': 0.34
        })

    def setup_layout(self):
        # Event Handler
        self.calculate_button.on_click(self.calculate_portfolios)
        self.optimize_button.on_click(self.optimize_portfolio)

        # Portfolio Tabs zusammenbauen
        portfolio_boxes = [config['box'] for config in self.portfolio_configs]
        portfolio_boxes.append(self.custom_portfolio['box'])

        self.portfolio_tabs.children = portfolio_boxes
        for i in range(4):
            self.portfolio_tabs.set_title(i, f'Portfolio {i+1}')
        self.portfolio_tabs.set_title(4, 'Custom')

        # Hauptlayout
        self.input_section = widgets.VBox([
            widgets.HTML('<h3>📊 Portfolio-Definition</h3>'),
            self.portfolio_tabs,
            self.calculate_button
        ])

        self.optimization_section = widgets.VBox([
            widgets.HTML('<h3>🎯 Portfolio-Optimierung</h3>'),
            self.optimization_type,
            widgets.HBox([self.constraint_type, self.constraint_value]),
            self.optimize_button,
            self.output_optimization
        ])

        self.results_section = widgets.Tab([
            self.output_summary,
            self.output_visualization,
            self.output_comparison
        ])
        self.results_section.set_title(0, 'Zusammenfassung')
        self.results_section.set_title(1, 'Visualisierung')
        self.results_section.set_title(2, 'Vergleich')

        self.main_layout = widgets.VBox([
            widgets.HBox([
                self.input_section,
                self.optimization_section
            ]),
            widgets.HTML('<h3>📈 Ergebnisse</h3>'),
            self.results_section
        ])

    def get_portfolio_weights(self):
        """Sammelt alle Portfolio-Gewichtungen"""
        all_configs = self.portfolio_configs + [self.custom_portfolio]
        portfolios = {}

        for i, config in enumerate(all_configs):
            name = config['name'].value
            weights = {
                'wind': config['wind'].value,
                'biomass': config['biomass'].value,
                'battery': config['battery'].value
            }
            # Nur Portfolios mit Summe ≈ 1 berücksichtigen
            if abs(sum(weights.values()) - 1.0) < 0.01:
                portfolios[name] = weights

        return portfolios

    def calculate_portfolio_metrics(self, weights, returns_data):
        """Berechnet NPV und Risiko für ein Portfolio"""
        # Portfolio-Returns für jede Simulation
        portfolio_returns = (
            returns_data['wind'] * weights['wind'] +
            returns_data['biomass'] * weights['biomass'] +
            returns_data['battery'] * weights['battery']
        )

        mean_return = portfolio_returns.mean()
        std_return = portfolio_returns.std()

        return mean_return, std_return, portfolio_returns

    def calculate_portfolios(self, button):
        """Berechnet alle Portfolios"""
        # Prüfe ob Simulationsdaten vorhanden
        if self.mc_simulator.simulation_results is None:
            with self.output_summary:
                clear_output()
                print("⚠️ Bitte führen Sie zuerst eine Monte-Carlo Simulation durch!")
            return

        # Sammle Portfolio-Definitionen
        portfolios = self.get_portfolio_weights()

        if not portfolios:
            with self.output_summary:
                clear_output()
                print("⚠️ Keine gültigen Portfolios definiert (Summe muss 100% sein)!")
            return

        # Berechne Metriken
        self.portfolio_results = {}

        for name, weights in portfolios.items():
            mean_npv, std_npv, returns = self.calculate_portfolio_metrics(
                weights, self.mc_simulator.simulation_results
            )

            self.portfolio_results[name] = {
                'weights': weights,
                'mean_npv': mean_npv,
                'std_npv': std_npv,
                'sharpe_ratio': mean_npv / std_npv if std_npv > 0 else 0,
                'returns': returns,
                'prob_positive': (returns > 0).mean() * 100,
                'var_95': np.percentile(returns, 5),
                'cvar_95': returns[returns <= np.percentile(returns, 5)].mean()
            }

        # Aktualisiere Anzeigen
        self.update_summary()
        self.update_visualization()
        self.update_comparison()

    def update_summary(self):
        """Zeigt Portfolio-Zusammenfassung"""
        with self.output_summary:
            clear_output(wait=True)

            # Erstelle Übersichtstabelle
            summary_data = []

            for name, metrics in self.portfolio_results.items():
                summary_data.append({
                    'Portfolio': name,
                    'NPV (Erwartung)': f"{metrics['mean_npv']:.2f}",
                    'Risiko (Std)': f"{metrics['std_npv']:.2f}",
                    'Sharpe Ratio': f"{metrics['sharpe_ratio']:.3f}",
                    'P(NPV>0)': f"{metrics['prob_positive']:.1f}%",
                    'VaR(95%)': f"{metrics['var_95']:.2f}",
                    'Wind': f"{metrics['weights']['wind']:.0%}",
                    'Biomasse': f"{metrics['weights']['biomass']:.0%}",
                    'Batterie': f"{metrics['weights']['battery']:.0%}"
                })

            df = pd.DataFrame(summary_data)

            # Styling
            display(widgets.HTML('<h4>Portfolio-Übersicht</h4>'))
            display(df.style\
                .background_gradient(subset=['NPV (Erwartung)'], cmap='RdYlGn')\
                .background_gradient(subset=['Risiko (Std)'], cmap='RdYlGn_r')\
                .background_gradient(subset=['Sharpe Ratio'], cmap='viridis'))

            # Beste Portfolios identifizieren
            display(widgets.HTML('<h4>Top-Portfolios</h4>'))

            best_npv = max(self.portfolio_results.items(), key=lambda x: x[1]['mean_npv'])
            best_sharpe = max(self.portfolio_results.items(), key=lambda x: x[1]['sharpe_ratio'])
            lowest_risk = min(self.portfolio_results.items(), key=lambda x: x[1]['std_npv'])

            print(f"📈 Höchster NPV: {best_npv[0]} ({best_npv[1]['mean_npv']:.2f} €/kW)")
            print(f"⚖️ Bestes Risiko-Rendite: {best_sharpe[0]} (Sharpe: {best_sharpe[1]['sharpe_ratio']:.3f})")
            print(f"🛡️ Geringstes Risiko: {lowest_risk[0]} (Std: {lowest_risk[1]['std_npv']:.2f} €/kW)")

    def update_visualization(self):
        """Erstellt interaktive Visualisierungen"""
        with self.output_visualization:
            clear_output(wait=True)

            # 1. Risiko-Rendite-Diagramm
            fig = make_subplots(
                rows=2, cols=2,
                subplot_titles=('Risiko-Rendite-Profil', 'Portfolio-Zusammensetzung',
                              'Verteilungen', 'Korrelation mit Einzeltechnologien'),
                specs=[[{"type": "scatter"}, {"type": "pie"}],
                       [{"type": "box"}, {"type": "scatter"}]]
            )

            # Subplot 1: Risiko-Rendite
            # Einzeltechnologien
            for tech in ['wind', 'biomass', 'battery']:
                tech_returns = self.mc_simulator.simulation_results[tech]
                fig.add_trace(
                    go.Scatter(
                        x=[tech_returns.std()],
                        y=[tech_returns.mean()],
                        mode='markers',
                        name=self.model.tech_params[tech]['name'],
                        marker=dict(size=15, color=colors[tech], symbol='circle'),
                        showlegend=True
                    ),
                    row=1, col=1
                )

            # Portfolios
            for name, metrics in self.portfolio_results.items():
                fig.add_trace(
                    go.Scatter(
                        x=[metrics['std_npv']],
                        y=[metrics['mean_npv']],
                        mode='markers+text',
                        name=name,
                        text=[name],
                        textposition='top center',
                        marker=dict(size=12, symbol='diamond'),
                        showlegend=True
                    ),
                    row=1, col=1
                )

            # Subplot 2: Portfolio-Zusammensetzung (für das beste Portfolio)
            best_portfolio = max(self.portfolio_results.items(),
                               key=lambda x: x[1]['sharpe_ratio'])

            fig.add_trace(
                go.Pie(
                    labels=['Wind', 'Biomasse', 'Batterie'],
                    values=[
                        best_portfolio[1]['weights']['wind'],
                        best_portfolio[1]['weights']['biomass'],
                        best_portfolio[1]['weights']['battery']
                    ],
                    marker_colors=[colors['wind'], colors['biomass'], colors['battery']],
                    title=best_portfolio[0]
                ),
                row=1, col=2
            )

            # Subplot 3: Verteilungen
            for name, metrics in self.portfolio_results.items():
                fig.add_trace(
                    go.Box(
                        y=metrics['returns'],
                        name=name,
                        boxmean=True
                    ),
                    row=2, col=1
                )

            # Subplot 4: Effizienzgrenze
            # Generiere zusätzliche Portfolios für glatte Kurve
            n_points = 50
            weights_range = np.linspace(0, 1, n_points)
            frontier_risk = []
            frontier_return = []

            for w1 in weights_range:
                for w2 in np.linspace(0, 1-w1, int((1-w1)*n_points)):
                    w3 = 1 - w1 - w2
                    if w3 >= 0:
                        weights = {'wind': w1, 'biomass': w2, 'battery': w3}
                        mean_ret, std_ret, _ = self.calculate_portfolio_metrics(
                            weights, self.mc_simulator.simulation_results
                        )
                        frontier_risk.append(std_ret)
                        frontier_return.append(mean_ret)

            # Efficient Frontier
            fig.add_trace(
                go.Scatter(
                    x=frontier_risk,
                    y=frontier_return,
                    mode='markers',
                    marker=dict(size=2, color='lightgray'),
                    name='Mögliche Portfolios',
                    showlegend=False
                ),
                row=2, col=2
            )

            # Layout
            fig.update_xaxes(title_text="Risiko (Std) [€/kW]", row=1, col=1)
            fig.update_yaxes(title_text="Erwarteter NPV [€/kW]", row=1, col=1)
            fig.update_yaxes(title_text="NPV [€/kW]", row=2, col=1)
            fig.update_xaxes(title_text="Risiko (Std) [€/kW]", row=2, col=2)
            fig.update_yaxes(title_text="Erwarteter NPV [€/kW]", row=2, col=2)

            fig.update_layout(height=800, showlegend=True,
                            title_text="Portfolio-Analyse Dashboard")
            fig.show()

    def update_comparison(self):
        """Vergleicht Portfolios mit Einzeltechnologien"""
        with self.output_comparison:
            clear_output(wait=True)

            # Vorbereitung der Daten
            comparison_data = []

            # Einzeltechnologien
            for tech in ['wind', 'biomass', 'battery']:
                tech_returns = self.mc_simulator.simulation_results[tech]
                comparison_data.append({
                    'Name': self.model.tech_params[tech]['name'],
                    'Typ': 'Einzeltechnologie',
                    'NPV': tech_returns.mean(),
                    'Risiko': tech_returns.std(),
                    'Sharpe': tech_returns.mean() / tech_returns.std(),
                    'P(NPV>0)': (tech_returns > 0).mean() * 100,
                    'VaR(95%)': np.percentile(tech_returns, 5)
                })

            # Portfolios
            for name, metrics in self.portfolio_results.items():
                comparison_data.append({
                    'Name': name,
                    'Typ': 'Portfolio',
                    'NPV': metrics['mean_npv'],
                    'Risiko': metrics['std_npv'],
                    'Sharpe': metrics['sharpe_ratio'],
                    'P(NPV>0)': metrics['prob_positive'],
                    'VaR(95%)': metrics['var_95']
                })

            df_comparison = pd.DataFrame(comparison_data)

            # Radar Chart
            fig = go.Figure()

            categories = ['NPV', 'Risiko⁻¹', 'Sharpe', 'P(NPV>0)', '-VaR(95%)']

            # Normalisiere Daten für Radar Chart (0-1 Skala)
            def normalize_column(col):
                return (col - col.min()) / (col.max() - col.min())

            for _, row in df_comparison.iterrows():
                values = [
                    normalize_column(df_comparison['NPV'])[row.name],
                    1 - normalize_column(df_comparison['Risiko'])[row.name],  # Invertiert
                    normalize_column(df_comparison['Sharpe'])[row.name],
                    row['P(NPV>0)'] / 100,  # Bereits in %
                    1 - normalize_column(df_comparison['VaR(95%)'].abs())[row.name]  # Invertiert
                ]

                fig.add_trace(go.Scatterpolar(
                    r=values,
                    theta=categories,
                    fill='toself',
                    name=row['Name'],
                    line=dict(width=2 if row['Typ'] == 'Portfolio' else 1,
                             dash='solid' if row['Typ'] == 'Portfolio' else 'dash')
                ))

            fig.update_layout(
                polar=dict(
                    radialaxis=dict(
                        visible=True,
                        range=[0, 1]
                    )),
                showlegend=True,
                title="Multidimensionaler Vergleich (normalisiert)",
                height=500
            )

            fig.show()

            # Tabelle
            display(widgets.HTML('<h4>Detaillierter Vergleich</h4>'))
            display(df_comparison.style.format({
                'NPV': '{:.2f}',
                'Risiko': '{:.2f}',
                'Sharpe': '{:.3f}',
                'P(NPV>0)': '{:.1f}%',
                'VaR(95%)': '{:.2f}'
            }).background_gradient(subset=['NPV', 'Sharpe'], cmap='RdYlGn')\
              .background_gradient(subset=['Risiko'], cmap='RdYlGn_r'))

    def optimize_portfolio(self, button):
        """Optimiert Portfolio basierend auf gewähltem Kriterium"""
        with self.output_optimization:
            clear_output(wait=True)

            if self.mc_simulator.simulation_results is None:
                print("⚠️ Keine Simulationsdaten vorhanden!")
                return

            print("🔄 Optimiere Portfolio...")

            # Optimierungsfunktion
            from scipy.optimize import minimize

            def objective(weights):
                w_dict = {'wind': weights[0], 'biomass': weights[1], 'battery': weights[2]}
                mean_npv, std_npv, _ = self.calculate_portfolio_metrics(
                    w_dict, self.mc_simulator.simulation_results
                )

                if self.optimization_type.value == 'Maximiere NPV':
                    return -mean_npv
                elif self.optimization_type.value == 'Minimiere Risiko':
                    return std_npv
                else:  # Maximiere Sharpe Ratio
                    return -mean_npv / std_npv if std_npv > 0 else 0

            # Constraints
            constraints = [{'type': 'eq', 'fun': lambda w: sum(w) - 1}]

            if self.constraint_type.value == 'Max. Risiko':
                def risk_constraint(weights):
                    w_dict = {'wind': weights[0], 'biomass': weights[1], 'battery': weights[2]}
                    _, std_npv, _ = self.calculate_portfolio_metrics(
                        w_dict, self.mc_simulator.simulation_results
                    )
                    return self.constraint_value.value - std_npv

                constraints.append({'type': 'ineq', 'fun': risk_constraint})

            elif self.constraint_type.value == 'Min. NPV':
                def npv_constraint(weights):
                    w_dict = {'wind': weights[0], 'biomass': weights[1], 'battery': weights[2]}
                    mean_npv, _, _ = self.calculate_portfolio_metrics(
                        w_dict, self.mc_simulator.simulation_results
                    )
                    return mean_npv - self.constraint_value.value

                constraints.append({'type': 'ineq', 'fun': npv_constraint})

            # Optimierung
            bounds = [(0, 1), (0, 1), (0, 1)]
            initial_guess = [1/3, 1/3, 1/3]

            result = minimize(objective, initial_guess, method='SLSQP',
                            bounds=bounds, constraints=constraints)

            if result.success:
                optimal_weights = {
                    'wind': result.x[0],
                    'biomass': result.x[1],
                    'battery': result.x[2]
                }

                mean_npv, std_npv, returns = self.calculate_portfolio_metrics(
                    optimal_weights, self.mc_simulator.simulation_results
                )

                print("✅ Optimierung erfolgreich!\n")
                print(f"Optimales Portfolio:")
                print(f"  Wind:     {optimal_weights['wind']:.1%}")
                print(f"  Biomasse: {optimal_weights['biomass']:.1%}")
                print(f"  Batterie: {optimal_weights['battery']:.1%}")
                print(f"\nErgebnisse:")
                print(f"  Erwarteter NPV: {mean_npv:.2f} €/kW")
                print(f"  Risiko (Std):   {std_npv:.2f} €/kW")
                print(f"  Sharpe Ratio:   {mean_npv/std_npv:.3f}")
                print(f"  P(NPV>0):       {(returns > 0).mean()*100:.1f}%")

                # Visualisierung
                fig = go.Figure(data=[
                    go.Bar(
                        x=['Wind', 'Biomasse', 'Batterie'],
                        y=[optimal_weights['wind'], optimal_weights['biomass'],
                           optimal_weights['battery']],
                        marker_color=[colors['wind'], colors['biomass'], colors['battery']],
                        text=[f"{w:.1%}" for w in optimal_weights.values()],
                        textposition='auto'
                    )
                ])

                fig.update_layout(
                    title=f'Optimales Portfolio ({self.optimization_type.value})',
                    yaxis_title='Anteil',
                    yaxis_tickformat='.0%',
                    height=600,
                    width=600
                )

                fig.show()

                # Füge optimales Portfolio zu Custom hinzu
                self.custom_portfolio['name'].value = 'Optimal'
                self.custom_portfolio['wind'].value = optimal_weights['wind']
                self.custom_portfolio['biomass'].value = optimal_weights['biomass']
                self.custom_portfolio['battery'].value = optimal_weights['battery']

            else:
                print("❌ Optimierung fehlgeschlagen!")
                print(f"Grund: {result.message}")

    def display(self):
        """Zeigt das komplette Widget an"""
        display(self.main_layout)

In [ ]:
# Cell 10: Portfolio Analyzer starten
portfolio_analyzer = PortfolioAnalyzer(model, mc_simulator)
portfolio_analyzer.display()

## Efficient Frontier Explorer

### Theoretischer Hintergrund

Die Efficient Frontier (Effizienzgrenze) zeigt alle Portfolios, die bei gegebenem Risiko die maximale Rendite bzw. bei gegebener Rendite das minimale Risiko aufweisen. Portfolios unterhalb dieser Grenze sind suboptimal.

### Praktische Anwendung

Mit dem interaktiven Tool können Sie:
1. Ein gewünschtes Risikoniveau wählen
2. Das optimale Portfolio für dieses Risiko ermitteln
3. Die resultierende Zusammensetzung analysieren

Dies unterstützt die Entscheidungsfindung basierend auf der individuellen Risikobereitschaft des Investors.

In [ ]:
# Cell 11: Efficient Frontier Exploration

class ImprovedEfficientFrontierExplorer:
    def __init__(self, portfolio_analyzer):
        self.analyzer = portfolio_analyzer
        self.frontier_data = None
        self.setup_widgets()
        self.calculate_frontier()

    def calculate_frontier(self):
        """Berechnet die komplette Efficient Frontier vorab"""
        if self.analyzer.mc_simulator.simulation_results is None:
            return

        # Generiere viele Portfolios für die Frontier
        n_portfolios = 1000
        self.frontier_data = {
            'weights': [],
            'returns': [],
            'risks': [],
            'sharpe': []
        }

        # Zufällige Portfolio-Gewichtungen
        for _ in range(n_portfolios):
            # Zufällige Gewichte die sich zu 1 summieren
            weights = np.random.random(3)
            weights = weights / weights.sum()

            weight_dict = {
                'wind': weights[0],
                'biomass': weights[1],
                'battery': weights[2]
            }

            mean_return, std_return, _ = self.analyzer.calculate_portfolio_metrics(
                weight_dict,
                self.analyzer.mc_simulator.simulation_results
            )

            self.frontier_data['weights'].append(weight_dict)
            self.frontier_data['returns'].append(mean_return)
            self.frontier_data['risks'].append(std_return)
            self.frontier_data['sharpe'].append(mean_return / std_return if std_return > 0 else 0)

    def setup_widgets(self):
        # Zwei Slider für bessere Kontrolle
        self.selection_mode = widgets.RadioButtons(
            options=['Nach Risiko wählen', 'Nach Rendite wählen'],
            value='Nach Risiko wählen',
            description='Modus:'
        )

        self.risk_slider = widgets.FloatSlider(
            value=400,
            min=200,
            max=800,
            step=10,
            description='Ziel-Risiko [€/kW]:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='60%')
        )

        self.return_slider = widgets.FloatSlider(
            value=1000,
            min=0,
            max=2000,
            step=50,
            description='Ziel-Rendite [€/kW]:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='60%'),
            disabled=True
        )

        # Info-Box
        self.info_box = widgets.HTML(
            value="""
            <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">
            <b>Was ist die Efficient Frontier?</b><br>
            Die Efficient Frontier zeigt alle optimalen Portfolios. Für jedes Risikoniveau
            zeigt sie das Portfolio mit der höchsten erwarteten Rendite.
            </div>
            """
        )

        self.output = widgets.Output()

        # Layout
        self.layout = widgets.VBox([
            widgets.HTML('<h3>Efficient Frontier Explorer</h3>'),
            self.info_box,
            self.selection_mode,
            self.risk_slider,
            self.return_slider,
            self.output
        ])

        # Event Handler
        self.selection_mode.observe(self.on_mode_change, 'value')
        self.risk_slider.observe(self.update_visualization, 'value')
        self.return_slider.observe(self.update_visualization, 'value')

    def on_mode_change(self, change):
        """Wechselt zwischen Risiko- und Rendite-basierter Auswahl"""
        if self.selection_mode.value == 'Nach Risiko wählen':
            self.risk_slider.disabled = False
            self.return_slider.disabled = True
        else:
            self.risk_slider.disabled = True
            self.return_slider.disabled = False
        self.update_visualization(None)

    def find_optimal_portfolio(self, target_value, mode='risk'):
        """Findet das optimale Portfolio für einen Zielwert"""
        if mode == 'risk':
            # Finde Portfolio mit Ziel-Risiko und maximaler Rendite
            candidates = []
            for i, risk in enumerate(self.frontier_data['risks']):
                if abs(risk - target_value) < 50:  # Toleranz
                    candidates.append({
                        'idx': i,
                        'weights': self.frontier_data['weights'][i],
                        'return': self.frontier_data['returns'][i],
                        'risk': risk,
                        'distance': abs(risk - target_value)
                    })

            if candidates:
                # Wähle das mit höchster Rendite
                best = max(candidates, key=lambda x: x['return'])
                return best
        else:
            # Finde Portfolio mit Ziel-Rendite und minimalem Risiko
            candidates = []
            for i, ret in enumerate(self.frontier_data['returns']):
                if abs(ret - target_value) < 100:  # Toleranz
                    candidates.append({
                        'idx': i,
                        'weights': self.frontier_data['weights'][i],
                        'return': ret,
                        'risk': self.frontier_data['risks'][i],
                        'distance': abs(ret - target_value)
                    })

            if candidates:
                # Wähle das mit geringstem Risiko
                best = min(candidates, key=lambda x: x['risk'])
                return best

        return None

    def update_visualization(self, change):
        """Aktualisiert die Visualisierung"""
        with self.output:
            clear_output(wait=True)

            if self.frontier_data is None:
                print("Keine Daten verfügbar. Bitte führen Sie zuerst eine Monte-Carlo Simulation durch.")
                return

            # Finde optimales Portfolio
            if self.selection_mode.value == 'Nach Risiko wählen':
                optimal = self.find_optimal_portfolio(self.risk_slider.value, 'risk')
            else:
                optimal = self.find_optimal_portfolio(self.return_slider.value, 'return')

            if optimal is None:
                print("Kein Portfolio für diesen Zielwert gefunden. Bitte anderen Wert wählen.")
                return

            # Erstelle Visualisierung
            fig = make_subplots(
                rows=1,
                cols=3,
                specs=[[{"type": "xy"}, {"type": "domain"}, {"type": "xy"}]],
                subplot_titles=(
                    'Efficient Frontier mit Ihrer Auswahl',
                    'Portfolio-Zusammensetzung',
                    'Vergleich mit Einzeltechnologien'
                ),
                column_widths=[0.4, 0.3, 0.3]
            )

            # 1. Efficient Frontier
            # Alle Portfolios
            fig.add_trace(
                go.Scatter(
                    x=self.frontier_data['risks'],
                    y=self.frontier_data['returns'],
                    mode='markers',
                    marker=dict(
                        size=3,
                        color=self.frontier_data['sharpe'],
                        colorscale='Viridis',
                        showscale=True,
                        colorbar=dict(title="Sharpe<br>Ratio", x=0.45)
                    ),
                    name='Alle Portfolios',
                    text=[f"Sharpe: {s:.2f}" for s in self.frontier_data['sharpe']],
                    hovertemplate='Risiko: %{x:.0f}<br>Rendite: %{y:.0f}<br>%{text}'
                ),
                row=1, col=1
            )

            # Gewähltes Portfolio
            fig.add_trace(
                go.Scatter(
                    x=[optimal['risk']],
                    y=[optimal['return']],
                    mode='markers+text',
                    marker=dict(size=20, color='red', symbol='star'),
                    text=['Ihre Wahl'],
                    textposition='top center',
                    name='Gewähltes Portfolio',
                    showlegend=False
                ),
                row=1, col=1
            )

            # Ziellinien
            if self.selection_mode.value == 'Nach Risiko wählen':
                fig.add_vline(
                    x=self.risk_slider.value,
                    line_dash="dash",
                    line_color="red",
                    opacity=0.5,
                    row=1, col=1
                )
            else:
                fig.add_hline(
                    y=self.return_slider.value,
                    line_dash="dash",
                    line_color="red",
                    opacity=0.5,
                    row=1, col=1
                )

            # 2. Portfolio-Zusammensetzung
            fig.add_trace(
                go.Pie(
                    labels=['Wind', 'Biomasse', 'Batterie'],
                    values=[
                        optimal['weights']['wind'],
                        optimal['weights']['biomass'],
                        optimal['weights']['battery']
                    ],
                    marker_colors=[colors['wind'], colors['biomass'], colors['battery']],
                    textinfo='label+percent',
                    textposition='auto'
                ),
                row=1, col=2
            )

            # 3. Vergleich
            # Einzeltechnologien
            single_techs_data = []
            for tech in ['wind', 'biomass', 'battery']:
                tech_returns = self.analyzer.mc_simulator.simulation_results[tech]
                single_techs_data.append({
                    'name': self.analyzer.model.tech_params[tech]['name'],
                    'return': tech_returns.mean(),
                    'risk': tech_returns.std(),
                    'type': 'Einzeltechnologie'
                })

            # Portfolio
            single_techs_data.append({
                'name': 'Gewähltes\nPortfolio',
                'return': optimal['return'],
                'risk': optimal['risk'],
                'type': 'Portfolio'
            })

            df_compare = pd.DataFrame(single_techs_data)

            # Balkendiagramm
            fig.add_trace(
                go.Bar(
                    x=df_compare['name'],
                    y=df_compare['return'],
                    name='Rendite',
                    marker_color='lightblue',
                    yaxis='y3'
                ),
                row=1, col=3
            )

            fig.add_trace(
                go.Bar(
                    x=df_compare['name'],
                    y=df_compare['risk'],
                    name='Risiko',
                    marker_color='lightcoral',
                    yaxis='y4',
                    opacity=0.7
                ),
                row=1, col=3
            )

            # Layout
            fig.update_xaxes(title_text="Risiko (Std.Abw.) [€/kW]", row=1, col=1)
            fig.update_yaxes(title_text="Erwartete Rendite (NPV) [€/kW]", row=1, col=1)

            # Zwei y-Achsen für Vergleich
            fig.update_layout(
                yaxis3=dict(
                    anchor="x3",
                    overlaying="y4",
                    side="left",
                    title="Rendite [€/kW]"
                ),
                yaxis4=dict(
                    anchor="x3",
                    side="right",
                    title="Risiko [€/kW]"
                )
            )

            fig.update_layout(
                height=500,
                showlegend=True,
                title_text=f"Portfolio-Analyse: {self.selection_mode.value}"
            )

            fig.show()

            # Detaillierte Informationen
            print("\nPortfolio-Details:")
            print("-" * 50)
            print(f"Zusammensetzung:")
            print(f"  Wind:     {optimal['weights']['wind']:.1%}")
            print(f"  Biomasse: {optimal['weights']['biomass']:.1%}")
            print(f"  Batterie: {optimal['weights']['battery']:.1%}")
            print(f"\nKennzahlen:")
            print(f"  Erwartete Rendite: {optimal['return']:.2f} €/kW")
            print(f"  Risiko (Std.Abw.): {optimal['risk']:.2f} €/kW")
            print(f"  Sharpe Ratio:      {optimal['return']/optimal['risk']:.3f}")

            # Vergleich mit Einzeltechnologien
            print("\nVergleich mit Einzeltechnologien:")
            for tech_data in single_techs_data[:-1]:  # Ohne Portfolio
                print(f"  {tech_data['name']}: Rendite={tech_data['return']:.0f}, Risiko={tech_data['risk']:.0f}")

            # Interpretation
            print("\nInterpretation:")
            if optimal['risk'] < min([d['risk'] for d in single_techs_data[:-1]]):
                print("✓ Das Portfolio hat ein geringeres Risiko als alle Einzeltechnologien (Diversifikationseffekt)")
            if optimal['return'] > min([d['return'] for d in single_techs_data[:-1]]):
                print("✓ Das Portfolio hat eine höhere Rendite als die schlechteste Einzeltechnologie")

    def display(self):
        display(self.layout)
        self.update_visualization(None)

# Explorer anzeigen
if hasattr(portfolio_analyzer, 'mc_simulator') and portfolio_analyzer.mc_simulator.simulation_results is not None:
    improved_frontier = ImprovedEfficientFrontierExplorer(portfolio_analyzer)
    improved_frontier.display()
else:
    print("Bitte führen Sie zuerst die Monte-Carlo Simulation und Portfolio-Berechnung durch!")

## Szenarioanalyse

### Zweck der Szenarioanalyse

In der Realität entwickeln sich Marktparameter nicht isoliert, sondern in konsistenten Mustern. Die Szenarioanalyse untersucht die Robustheit der Investitionsentscheidung unter verschiedenen plausiblen Zukunftsentwicklungen.

### Vordefinierte Szenarien

1. **Optimistisch**: Günstige Marktentwicklung mit hohen Strompreisen
2. **Realistisch**: Fortschreibung aktueller Trends
3. **Pessimistisch**: Ungünstige Entwicklung mit niedrigen Erlösen
4. **Volatil**: Hohe Marktunsicherheit
5. **Dekarbonisierung**: CO2-Bepreisung treibt Transformation

### Vergleichsanalyse

Die Ergebnisse zeigen:
- Welche Technologie in welchem Szenario optimal ist
- Wie robust die Investitionsentscheidung über verschiedene Szenarien ist
- Welche Parameter die größte Unsicherheit verursachen

### Sensitivität über Szenarien

Diese Analyse visualisiert, wie sich die NPV-Werte mit den Szenarioparametern ändern. Dies hilft, kritische Schwellenwerte und Kipppunkte zu identifizieren.

In [ ]:
# Cell 12: Szenario-Manager

import json
from datetime import datetime

class ScenarioManager:
    def __init__(self, model, parameter_explorer, mc_simulator, portfolio_analyzer):
        self.model = model
        self.parameter_explorer = parameter_explorer
        self.mc_simulator = mc_simulator
        self.portfolio_analyzer = portfolio_analyzer
        self.scenarios = {}
        self.setup_predefined_scenarios()
        self.setup_widgets()
        self.setup_layout()

    def setup_predefined_scenarios(self):
        """Definiert Standard-Szenarien"""
        self.predefined_scenarios = {
            'Optimistisch': {
                'description': 'Hohe Strompreise, niedrige Kosten',
                'parameters': {
                    'elec_price_mean': 0.085,
                    'elec_price_std': 0.002,
                    'bio_price_mean': 55,
                    'bio_price_std': 8,
                    'discount_rate': 0.04
                },
                'color': 'green'
            },
            'Realistisch': {
                'description': 'Durchschnittliche Marktbedingungen',
                'parameters': {
                    'elec_price_mean': 0.06976,
                    'elec_price_std': 0.00244,
                    'bio_price_mean': 70.92,
                    'bio_price_std': 11.76,
                    'discount_rate': 0.05
                },
                'color': 'blue'
            },
            'Pessimistisch': {
                'description': 'Niedrige Strompreise, hohe Kosten',
                'parameters': {
                    'elec_price_mean': 0.055,
                    'elec_price_std': 0.003,
                    'bio_price_mean': 85,
                    'bio_price_std': 15,
                    'discount_rate': 0.06
                },
                'color': 'red'
            },
            'Volatil': {
                'description': 'Hohe Preisschwankungen',
                'parameters': {
                    'elec_price_mean': 0.07,
                    'elec_price_std': 0.008,
                    'bio_price_mean': 70,
                    'bio_price_std': 20,
                    'discount_rate': 0.05
                },
                'color': 'orange'
            },
            'Dekarbonisierung': {
                'description': 'CO2-Preis treibt Strompreise',
                'parameters': {
                    'elec_price_mean': 0.09,
                    'elec_price_std': 0.004,
                    'bio_price_mean': 65,
                    'bio_price_std': 10,
                    'discount_rate': 0.045
                },
                'color': 'purple'
            }
        }

    def setup_widgets(self):
        # Szenario-Auswahl
        self.scenario_selector = widgets.Dropdown(
            options=list(self.predefined_scenarios.keys()) + ['Custom'],
            value='Realistisch',
            description='Szenario:',
            style={'description_width': '80px'}
        )

        # Custom Szenario Name
        self.custom_name = widgets.Text(
            value='Mein Szenario',
            description='Name:',
            style={'description_width': '80px'},
            layout=widgets.Layout(width='300px')
        )

        # Parameter-Eingaben für Custom Szenario
        self.custom_params = {
            'elec_price_mean': widgets.FloatSlider(
                value=0.07, min=0.03, max=0.12, step=0.001,
                description='Strom μ [€/kWh]:', style={'description_width': '120px'}
            ),
            'elec_price_std': widgets.FloatSlider(
                value=0.003, min=0.001, max=0.01, step=0.0001,
                description='Strom σ [€/kWh]:', style={'description_width': '120px'}
            ),
            'bio_price_mean': widgets.FloatSlider(
                value=70, min=30, max=120, step=1,
                description='Biomasse μ [€/t]:', style={'description_width': '120px'}
            ),
            'bio_price_std': widgets.FloatSlider(
                value=12, min=2, max=30, step=1,
                description='Biomasse σ [€/t]:', style={'description_width': '120px'}
            ),
            'discount_rate': widgets.FloatSlider(
                value=0.05, min=0.01, max=0.15, step=0.01,
                description='Diskontrate:', style={'description_width': '120px'}
            )
        }

        # Buttons
        self.apply_button = widgets.Button(
            description='Szenario anwenden',
            button_style='primary',
            icon='check'
        )

        self.save_button = widgets.Button(
            description='Szenario speichern',
            button_style='success',
            icon='save'
        )

        self.compare_button = widgets.Button(
            description='Szenarien vergleichen',
            button_style='info',
            icon='chart-bar'
        )

        self.run_all_button = widgets.Button(
            description='Alle Szenarien berechnen',
            button_style='warning',
            icon='play-circle'
        )

        # Szenario-Auswahl für Vergleich
        self.compare_scenarios = widgets.SelectMultiple(
            options=list(self.predefined_scenarios.keys()),
            value=['Optimistisch', 'Realistisch', 'Pessimistisch'],
            description='Vergleiche:',
            style={'description_width': '80px'},
            layout=widgets.Layout(height='120px')
        )

        # Output Areas
        self.output_status = widgets.Output()
        self.output_current = widgets.Output()
        self.output_comparison = widgets.Output()
        self.output_sensitivity = widgets.Output()

        # Progress
        self.progress = widgets.IntProgress(
            value=0, min=0, max=100,
            description='Fortschritt:',
            style={'description_width': '80px'}
        )

    def setup_layout(self):
        # Event Handler
        self.scenario_selector.observe(self.on_scenario_change, 'value')
        self.apply_button.on_click(self.apply_scenario)
        self.save_button.on_click(self.save_scenario)
        self.compare_button.on_click(self.compare_scenarios_handler)
        self.run_all_button.on_click(self.run_all_scenarios)

        # Layout
        self.scenario_box = widgets.VBox([
            widgets.HTML('<h3>📋 Szenario-Auswahl</h3>'),
            self.scenario_selector,
            self.output_current
        ])

        self.custom_box = widgets.VBox([
            widgets.HTML('<h3>⚙️ Custom Szenario</h3>'),
            self.custom_name,
            *list(self.custom_params.values()),
            widgets.HBox([self.apply_button, self.save_button])
        ])

        self.comparison_box = widgets.VBox([
            widgets.HTML('<h3>📊 Szenario-Vergleich</h3>'),
            self.compare_scenarios,
            widgets.HBox([self.compare_button, self.run_all_button]),
            self.progress,
            self.output_status
        ])

        self.results_tabs = widgets.Tab([
            self.output_comparison,
            self.output_sensitivity
        ])
        self.results_tabs.set_title(0, 'Vergleich')
        self.results_tabs.set_title(1, 'Sensitivität')

        self.main_layout = widgets.VBox([
            widgets.HBox([
                self.scenario_box,
                self.custom_box,
                self.comparison_box
            ]),
            widgets.HTML('<h3>📈 Ergebnisse</h3>'),
            self.results_tabs
        ])

        # Initial update
        self.on_scenario_change(None)

    def on_scenario_change(self, change):
        """Aktualisiert Anzeige bei Szenario-Wechsel"""
        with self.output_current:
            clear_output(wait=True)

            current = self.scenario_selector.value

            if current in self.predefined_scenarios:
                scenario = self.predefined_scenarios[current]
                display(widgets.HTML(f"<b>{current}:</b> {scenario['description']}"))

                # Parameter-Tabelle
                params = scenario['parameters']
                df = pd.DataFrame([
                    {'Parameter': 'Strompreis',
                     'Mittelwert': f"{params['elec_price_mean']:.4f} €/kWh",
                     'Std.Abw.': f"{params['elec_price_std']:.4f} €/kWh"},
                    {'Parameter': 'Biomassepreis',
                     'Mittelwert': f"{params['bio_price_mean']:.1f} €/t",
                     'Std.Abw.': f"{params['bio_price_std']:.1f} €/t"},
                    {'Parameter': 'Diskontrate',
                     'Mittelwert': f"{params['discount_rate']:.1%}",
                     'Std.Abw.': '-'}
                ])
                display(df.style.hide(axis='index'))

            else:  # Custom
                display(widgets.HTML("<b>Custom Szenario</b>"))
                display(widgets.HTML("Passen Sie die Parameter rechts an."))

        # Custom-Parameter ein-/ausblenden
        self.custom_box.layout.visibility = 'visible' if current == 'Custom' else 'hidden'

    def apply_scenario(self, button):
        """Wendet ausgewähltes Szenario auf alle Tools an"""
        current = self.scenario_selector.value

        if current in self.predefined_scenarios:
            params = self.predefined_scenarios[current]['parameters']
        else:  # Custom
            params = {key: widget.value for key, widget in self.custom_params.items()}

        # Update Parameter Explorer
        self.parameter_explorer.elec_price_slider.value = params['elec_price_mean']
        self.parameter_explorer.bio_price_slider.value = params['bio_price_mean']
        self.parameter_explorer.discount_slider.value = params['discount_rate']

        # Update Monte Carlo Simulator
        self.mc_simulator.elec_mean.value = params['elec_price_mean']
        self.mc_simulator.elec_std.value = params['elec_price_std']
        self.mc_simulator.bio_mean.value = params['bio_price_mean']
        self.mc_simulator.bio_std.value = params['bio_price_std']
        self.mc_simulator.discount_rate.value = params['discount_rate']

        with self.output_status:
            clear_output()
            print(f"✅ Szenario '{current}' wurde angewendet!")

    def save_scenario(self, button):
        """Speichert Custom Szenario"""
        if self.scenario_selector.value != 'Custom':
            return

        name = self.custom_name.value
        params = {key: widget.value for key, widget in self.custom_params.items()}

        # Speichere in Datei
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f'scenario_{name.replace(" ", "_")}_{timestamp}.json'

        scenario_data = {
            'name': name,
            'timestamp': timestamp,
            'parameters': params,
            'description': 'Custom scenario'
        }

        with open(filename, 'w') as f:
            json.dump(scenario_data, f, indent=4)

        with self.output_status:
            clear_output()
            print(f"✅ Szenario gespeichert: {filename}")

    def run_all_scenarios(self, button):
        """Führt Simulationen für alle ausgewählten Szenarien durch"""
        scenarios_to_run = list(self.compare_scenarios.value)
        n_scenarios = len(scenarios_to_run)

        if n_scenarios == 0:
            with self.output_status:
                clear_output()
                print("⚠️ Bitte wählen Sie mindestens ein Szenario aus!")
            return

        with self.output_status:
            clear_output()
            print(f"🚀 Starte Berechnung für {n_scenarios} Szenarien...")

        # Speichere aktuelle Einstellungen
        original_params = {
            'elec_mean': self.mc_simulator.elec_mean.value,
            'elec_std': self.mc_simulator.elec_std.value,
            'bio_mean': self.mc_simulator.bio_mean.value,
            'bio_std': self.mc_simulator.bio_std.value,
            'discount': self.mc_simulator.discount_rate.value
        }

        # Ergebnisse sammeln
        self.scenarios = {}

        for i, scenario_name in enumerate(scenarios_to_run):
            # Update Progress
            self.progress.value = int((i / n_scenarios) * 100)

            # Setze Parameter
            params = self.predefined_scenarios[scenario_name]['parameters']
            self.mc_simulator.elec_mean.value = params['elec_price_mean']
            self.mc_simulator.elec_std.value = params['elec_price_std']
            self.mc_simulator.bio_mean.value = params['bio_price_mean']
            self.mc_simulator.bio_std.value = params['bio_price_std']
            self.mc_simulator.discount_rate.value = params['discount_rate']

            # Führe Simulation durch
            self.mc_simulator.run_simulation(None)

            # Speichere Ergebnisse
            self.scenarios[scenario_name] = {
                'parameters': params,
                'results': self.mc_simulator.simulation_results.copy(),
                'statistics': {
                    'wind': {
                        'mean': self.mc_simulator.simulation_results['wind'].mean(),
                        'std': self.mc_simulator.simulation_results['wind'].std(),
                        'prob_positive': (self.mc_simulator.simulation_results['wind'] > 0).mean() * 100
                    },
                    'biomass': {
                        'mean': self.mc_simulator.simulation_results['biomass'].mean(),
                        'std': self.mc_simulator.simulation_results['biomass'].std(),
                        'prob_positive': (self.mc_simulator.simulation_results['biomass'] > 0).mean() * 100
                    },
                    'battery': {
                        'mean': self.mc_simulator.simulation_results['battery'].mean(),
                        'std': self.mc_simulator.simulation_results['battery'].std(),
                        'prob_positive': (self.mc_simulator.simulation_results['battery'] > 0).mean() * 100
                    }
                }
            }

        # Progress abschließen
        self.progress.value = 100

        # Stelle ursprüngliche Parameter wieder her
        for key, value in original_params.items():
            getattr(self.mc_simulator, key).value = value

        with self.output_status:
            print(f"✅ Alle {n_scenarios} Szenarien berechnet!")

        # Automatisch Vergleich anzeigen
        self.show_comparison()

    def compare_scenarios_handler(self, button):
        """Handler für Vergleichs-Button"""
        if not self.scenarios:
            with self.output_status:
                clear_output()
                print("⚠️ Bitte berechnen Sie zuerst die Szenarien!")
            return

        self.show_comparison()
        self.show_sensitivity_analysis()

    def show_comparison(self):
        """Zeigt Szenario-Vergleich"""
        with self.output_comparison:
            clear_output(wait=True)

            if not self.scenarios:
                return

            # Vergleichstabelle
            comparison_data = []

            for scenario_name, data in self.scenarios.items():
                for tech in ['wind', 'biomass', 'battery']:
                    comparison_data.append({
                        'Szenario': scenario_name,
                        'Technologie': self.model.tech_params[tech]['name'],
                        'NPV (Mittel)': data['statistics'][tech]['mean'],
                        'Risiko (Std)': data['statistics'][tech]['std'],
                        'P(NPV>0)': data['statistics'][tech]['prob_positive']
                    })

            df_comparison = pd.DataFrame(comparison_data)

            # Visualisierung
            fig = make_subplots(
                rows=2, cols=2,
                subplot_titles=('NPV nach Szenario', 'Risiko nach Szenario',
                              'Wahrscheinlichkeit NPV>0', 'Szenario-Ranking'),
                specs=[[{"type": "bar"}, {"type": "bar"}],
                       [{"type": "bar"}, {"type": "scatter"}]]
            )

            # Subplot 1: NPV nach Szenario
            for tech in ['wind', 'biomass', 'battery']:
                tech_data = df_comparison[df_comparison['Technologie'] == self.model.tech_params[tech]['name']]
                fig.add_trace(
                    go.Bar(
                        x=tech_data['Szenario'],
                        y=tech_data['NPV (Mittel)'],
                        name=self.model.tech_params[tech]['name'],
                        marker_color=colors[tech]
                    ),
                    row=1, col=1
                )

            # Subplot 2: Risiko nach Szenario
            for tech in ['wind', 'biomass', 'battery']:
                tech_data = df_comparison[df_comparison['Technologie'] == self.model.tech_params[tech]['name']]
                fig.add_trace(
                    go.Bar(
                        x=tech_data['Szenario'],
                        y=tech_data['Risiko (Std)'],
                        name=self.model.tech_params[tech]['name'],
                        marker_color=colors[tech],
                        showlegend=False
                    ),
                    row=1, col=2
                )

            # Subplot 3: Wahrscheinlichkeit NPV>0
            for tech in ['wind', 'biomass', 'battery']:
                tech_data = df_comparison[df_comparison['Technologie'] == self.model.tech_params[tech]['name']]
                fig.add_trace(
                    go.Bar(
                        x=tech_data['Szenario'],
                        y=tech_data['P(NPV>0)'],
                        name=self.model.tech_params[tech]['name'],
                        marker_color=colors[tech],
                        showlegend=False
                    ),
                    row=2, col=1
                )

            # Subplot 4: Szenario-Ranking (Bubble Chart)
            for scenario in self.scenarios.keys():
                scenario_data = df_comparison[df_comparison['Szenario'] == scenario]
                avg_npv = scenario_data['NPV (Mittel)'].mean()
                avg_risk = scenario_data['Risiko (Std)'].mean()
                avg_prob = scenario_data['P(NPV>0)'].mean()

                color = self.predefined_scenarios[scenario]['color'] if scenario in self.predefined_scenarios else 'gray'

                fig.add_trace(
                    go.Scatter(
                        x=[avg_risk],
                        y=[avg_npv],
                        mode='markers+text',
                        marker=dict(
                            size=avg_prob/2,  # Größe basiert auf Erfolgswahrscheinlichkeit
                            color=color,
                            line=dict(width=2, color='white')
                        ),
                        text=[scenario],
                        textposition='top center',
                        name=scenario,
                        showlegend=False
                    ),
                    row=2, col=2
                )

            # Layout
            fig.update_xaxes(title_text="Szenario", row=1, col=1)
            fig.update_xaxes(title_text="Szenario", row=1, col=2)
            fig.update_xaxes(title_text="Szenario", row=2, col=1)
            fig.update_xaxes(title_text="Durchschn. Risiko [€/kW]", row=2, col=2)

            fig.update_yaxes(title_text="NPV [€/kW]", row=1, col=1)
            fig.update_yaxes(title_text="Std.Abw. [€/kW]", row=1, col=2)
            fig.update_yaxes(title_text="Wahrscheinlichkeit [%]", row=2, col=1)
            fig.update_yaxes(title_text="Durchschn. NPV [€/kW]", row=2, col=2)

            fig.update_layout(height=800, showlegend=True, title_text="Szenario-Vergleich")
            fig.show()

            # Zusammenfassende Tabelle
            display(widgets.HTML('<h4>Detaillierte Ergebnisse</h4>'))
            pivot_table = df_comparison.pivot_table(
                index='Technologie',
                columns='Szenario',
                values='NPV (Mittel)',
                aggfunc='mean'
            )

            display(pivot_table.style.format('{:.2f}')\
                    .background_gradient(cmap='RdYlGn', axis=1))

    def show_sensitivity_analysis(self):
        """Zeigt Sensitivitätsanalyse über Szenarien"""
        with self.output_sensitivity:
            clear_output(wait=True)

            if not self.scenarios:
                return

            # Sammle Parameter und Ergebnisse
            param_data = []

            for scenario_name, data in self.scenarios.items():
                params = data['parameters']
                stats = data['statistics']

                param_data.append({
                    'scenario': scenario_name,
                    'elec_price': params['elec_price_mean'],
                    'bio_price': params['bio_price_mean'],
                    'discount': params['discount_rate'],
                    'wind_npv': stats['wind']['mean'],
                    'biomass_npv': stats['biomass']['mean'],
                    'battery_npv': stats['battery']['mean']
                })

            df_params = pd.DataFrame(param_data)

            # Korrelationsanalyse
            fig = make_subplots(
                rows=2, cols=3,
                subplot_titles=('Wind: Strompreis-Sensitivität',
                              'Biomasse: Strompreis-Sensitivität',
                              'Batterie: Strompreis-Sensitivität',
                              'Wind: Diskontrate-Sensitivität',
                              'Biomasse: Biomassepreis-Sensitivität',
                              'Portfolio: Gesamtsensitivität')
            )

            # Strompreis-Sensitivität
            for col, tech in enumerate(['wind', 'biomass', 'battery'], 1):
                fig.add_trace(
                    go.Scatter(
                        x=df_params['elec_price'],
                        y=df_params[f'{tech}_npv'],
                        mode='markers+lines',
                        marker=dict(size=10, color=colors[tech]),
                        line=dict(color=colors[tech]),
                        text=df_params['scenario'],
                        name=self.model.tech_params[tech]['name']
                    ),
                    row=1, col=col
                )

            # Diskontrate-Sensitivität (Wind)
            fig.add_trace(
                go.Scatter(
                    x=df_params['discount'],
                    y=df_params['wind_npv'],
                    mode='markers+lines',
                    marker=dict(size=10, color=colors['wind']),
                    text=df_params['scenario'],
                    showlegend=False
                ),
                row=2, col=1
            )

            # Biomassepreis-Sensitivität
            fig.add_trace(
                go.Scatter(
                    x=df_params['bio_price'],
                    y=df_params['biomass_npv'],
                    mode='markers+lines',
                    marker=dict(size=10, color=colors['biomass']),
                    text=df_params['scenario'],
                    showlegend=False
                ),
                row=2, col=2
            )

            # Portfolio-Sensitivität (3D-artig)
            # Durchschnittliches Portfolio (gleiche Gewichtung)
            df_params['portfolio_npv'] = (df_params['wind_npv'] +
                                         df_params['biomass_npv'] +
                                         df_params['battery_npv']) / 3

            # Bubble chart mit 3 Dimensionen
            fig.add_trace(
                go.Scatter(
                    x=df_params['elec_price'],
                    y=df_params['portfolio_npv'],
                    mode='markers+text',
                    marker=dict(
                        size=df_params['bio_price']/5,  # Größe = Biomassepreis
                        color=df_params['discount'],     # Farbe = Diskontrate
                        colorscale='Viridis',
                        showscale=True,
                        colorbar=dict(title="Diskontrate")
                    ),
                    text=df_params['scenario'],
                    textposition='top center',
                    showlegend=False
                ),
                row=2, col=3
            )

            # Achsenbeschriftungen
            fig.update_xaxes(title_text="Strompreis [€/kWh]", row=1)
            fig.update_xaxes(title_text="Diskontrate", row=2, col=1)
            fig.update_xaxes(title_text="Biomassepreis [€/t]", row=2, col=2)
            fig.update_xaxes(title_text="Strompreis [€/kWh]", row=2, col=3)

            fig.update_yaxes(title_text="NPV [€/kW]", row=1)
            fig.update_yaxes(title_text="NPV [€/kW]", row=2)

            fig.update_layout(height=700, showlegend=True,
                            title_text="Sensitivitätsanalyse über Szenarien")
            fig.show()

            # Elastizitäten berechnen
            display(widgets.HTML('<h4>Parameter-Elastizitäten</h4>'))

            # Vereinfachte Elastizitätsberechnung
            elasticities = {}

            for tech in ['wind', 'biomass', 'battery']:
                # Strompreis-Elastizität
                elec_elasticity = np.corrcoef(df_params['elec_price'],
                                             df_params[f'{tech}_npv'])[0, 1]
                elasticities[self.model.tech_params[tech]['name']] = {
                    'Strompreis': elec_elasticity
                }

                if tech == 'biomass':
                    # Biomassepreis-Elastizität
                    bio_elasticity = np.corrcoef(df_params['bio_price'],
                                                df_params['biomass_npv'])[0, 1]
                    elasticities['Biomass']['Biomassepreis'] = bio_elasticity

            elast_df = pd.DataFrame(elasticities).T
            display(elast_df.style.format('{:.3f}')\
                    .background_gradient(cmap='coolwarm', center=0))

            display(widgets.HTML("""
            <p><b>Interpretation:</b><br>
            • Positive Werte: NPV steigt mit steigendem Parameter<br>
            • Negative Werte: NPV sinkt mit steigendem Parameter<br>
            • |Wert| > 0.7: Starke Abhängigkeit</p>
            """))

    def display(self):
        """Zeigt das komplette Widget an"""
        display(self.main_layout)

# Cell 13: Szenario-Manager starten
scenario_manager = ScenarioManager(model, explorer, mc_simulator, portfolio_analyzer)
scenario_manager.display()

## Automatische Berichtserstellung

### Dokumentation der Ergebnisse

Der Report-Generator erstellt eine strukturierte Zusammenfassung aller durchgeführten Analysen. Dies umfasst:

- Executive Summary mit Kernaussagen
- Detaillierte Ergebnisse je Szenario
- Empfehlungen basierend auf der Gesamtanalyse

### Verwendung

Der generierte Bericht kann als Grundlage für:
- Die Abgabe der Case Study
- Präsentationen der Ergebnisse
- Weitere Diskussionen und Analysen

dienen. Das standardisierte Format gewährleistet Vollständigkeit und Nachvollziehbarkeit.

In [ ]:
# Cell 14: Automatischer Report-Generator

class ScenarioReportGenerator:
    def __init__(self, scenario_manager):
        self.scenario_manager = scenario_manager
        self.setup_widgets()

    def setup_widgets(self):
        self.report_button = widgets.Button(
            description='Report erstellen',
            button_style='primary',
            icon='file-pdf'
        )

        self.output = widgets.Output()

        self.layout = widgets.VBox([
            widgets.HTML('<h3>📄 Szenario-Report Generator</h3>'),
            widgets.HTML('<p>Erstellt eine Zusammenfassung aller berechneten Szenarien</p>'),
            self.report_button,
            self.output
        ])

        self.report_button.on_click(self.generate_report)

    def generate_report(self, button):
        """Generiert umfassenden Report"""
        with self.output:
            clear_output(wait=True)

            if not self.scenario_manager.scenarios:
                print("⚠️ Keine Szenarien berechnet!")
                return

            # Report-Struktur
            report = []
            report.append("=" * 80)
            report.append("ENERGIEMANAGEMENT CASE STUDY - SZENARIO-ANALYSE REPORT")
            report.append("=" * 80)
            report.append(f"Erstellt am: {datetime.now().strftime('%d.%m.%Y %H:%M')}")
            report.append("")

            # Executive Summary
            report.append("EXECUTIVE SUMMARY")
            report.append("-" * 40)

            # Finde beste Technologie pro Szenario
            best_by_scenario = {}
            for scenario, data in self.scenario_manager.scenarios.items():
                best_tech = max(data['statistics'].items(),
                              key=lambda x: x[1]['mean'])[0]
                best_by_scenario[scenario] = best_tech

            report.append("Empfohlene Technologie je Szenario:")
            for scenario, tech in best_by_scenario.items():
                report.append(f"  • {scenario}: {self.scenario_manager.model.tech_params[tech]['name']}")

            report.append("")

            # Robusteste Option
            tech_scores = {'wind': 0, 'biomass': 0, 'battery': 0}
            for tech in best_by_scenario.values():
                tech_scores[tech] += 1

            most_robust = max(tech_scores.items(), key=lambda x: x[1])[0]
            report.append(f"Robusteste Option über alle Szenarien: {self.scenario_manager.model.tech_params[most_robust]['name']}")

            # Detaillierte Ergebnisse
            report.append("\n" + "=" * 80)
            report.append("DETAILLIERTE ERGEBNISSE")
            report.append("=" * 80)

            for scenario, data in self.scenario_manager.scenarios.items():
                report.append(f"\nSzenario: {scenario}")
                report.append("-" * 40)

                # Parameter
                params = data['parameters']
                report.append("Parameter:")
                report.append(f"  Strompreis: {params['elec_price_mean']:.4f} ± {params['elec_price_std']:.4f} €/kWh")
                report.append(f"  Biomassepreis: {params['bio_price_mean']:.1f} ± {params['bio_price_std']:.1f} €/t")
                report.append(f"  Diskontrate: {params['discount_rate']:.1%}")

                # Ergebnisse
                report.append("\nErgebnisse:")
                for tech, stats in data['statistics'].items():
                    tech_name = self.scenario_manager.model.tech_params[tech]['name']
                    report.append(f"  {tech_name}:")
                    report.append(f"    NPV: {stats['mean']:.2f} ± {stats['std']:.2f} €/kW")
                    report.append(f"    P(NPV>0): {stats['prob_positive']:.1f}%")

            # Speichern
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f'scenario_report_{timestamp}.txt'

            with open(filename, 'w', encoding='utf-8') as f:
                f.write('\n'.join(report))

            print(f"✅ Report erstellt: {filename}")
            print("\nVorschau:")
            print("-" * 40)
            for line in report[:20]:
                print(line)
            print("...")

    def display(self):
        display(self.layout)

# Report Generator anzeigen
report_generator = ScenarioReportGenerator(scenario_manager)
report_generator.display()

## Zusammenfassung und Schlussfolgerungen

### Gelernte Konzepte

In dieser Case Study haben Sie:
1. NPV-Berechnungen unter Unsicherheit durchgeführt
2. Monte-Carlo-Simulationen zur Risikoanalyse eingesetzt
3. Portfolio-Optimierung angewendet
4. Szenarioanalysen für robuste Entscheidungen genutzt

### Praktische Implikationen

Die Analysen zeigen, dass Investitionsentscheidungen im Energiesektor komplex sind und von vielen unsicheren Faktoren abhängen. Die verwendeten Methoden helfen, diese Unsicherheiten zu quantifizieren und fundierte Entscheidungen zu treffen.

### Reflexionsfragen

- Welche Technologie würden Sie basierend auf Ihrer Analyse empfehlen?
- Wie beeinflusst Ihre Risikobereitschaft die Entscheidung?
- Welche zusätzlichen Faktoren sollten in der Praxis berücksichtigt werden?